Universe of fin

https://cvc5.github.io/docs/cvc5-1.0.2/options.html#additional-cvc5-options
ho-elim 
https://www.gdr-gpl.cnrs.fr/sites/default/files/documentsGPL/JourneesNationales/GPL2024/tourret_hosmt_user.pdf
ho-smt paper
https://nekoka-project.github.io/

In [138]:
import cvc5.pythonic as cvc5
cvc5.


In [139]:

import cvc5
from cvc5 import Kind

tm = cvc5.TermManager()
slv = cvc5.Solver(tm)
slv.setLogic("QF_NRAT")

real = tm.getRealSort()

# Variables
x = tm.mkConst(real, "x")
y = tm.mkConst(real, "y")

# Helper terms
two = tm.mkReal(2)
pi = tm.mkPi()
twopi = tm.mkTerm(Kind.MULT, two, pi)
ysq = tm.mkTerm(Kind.MULT, y, y)
sinx = tm.mkTerm(Kind.SINE, x)

# Formulas
x_gt_pi = tm.mkTerm(Kind.GT, x, pi)
x_lt_tpi = tm.mkTerm(Kind.LT, x, twopi)
ysq_lt_sinx = tm.mkTerm(Kind.LT, ysq, sinx)

slv.assertFormula(x_gt_pi)
slv.assertFormula(x_lt_tpi)
slv.assertFormula(ysq_lt_sinx)

print("cvc5 should report UNSAT")
print("Result from cvc5 is:", slv.checkSat())

AttributeError: module 'cvc5' has no attribute 'TermManager'

In [ ]:
from z3 import *
x = Real("x")
solve(x**0.5 < 0)
solve(x**0.5 == 0)
solve(x**0.5 >= 0)
prove(Implies(x >= 0, x**0.5 >= 0))
#prove(ForAll([x], Implies(x >= 0, x**0.5 >= 0)))

In [2]:
from z3 import *
print(get_version_string())
x = Real("x")
s = Solver()
s.add(Not(ForAll([x], Implies(x >= 0, x**0.5 >= 0))))
print(s.check())

s = Solver()
s.add(Exists([x], Not(Implies(x >= 0, x**0.5 >= 0))))
print(s.check())

s = Solver()
s.add(Not(Implies(x >= 0, x**0.5 >= 0)))
print(s.check())


4.13.0
unknown
unknown
unsat


In [23]:
!python3 -m pip install z3-solver --upgrade

Defaulting to user installation because normal site-packages is not writeable
  Using cached z3_solver-4.13.0.0-py2.py3-none-manylinux2014_x86_64.whl.metadata (757 bytes)
Using cached z3_solver-4.13.0.0-py2.py3-none-manylinux2014_x86_64.whl (57.3 MB)
  Attempting uninstall: z3-solver
    Found existing installation: z3-solver 4.12.4.0
    Uninstalling z3-solver-4.12.4.0:
      Successfully uninstalled z3-solver-4.12.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
claripy 9.2.94 requires z3-solver==4.10.2.0, but you have z3-solver 4.13.0.0 which is incompatible.
idp-engine 0.11.1 requires z3-solver==4.12.4.0, but you have z3-solver 4.13.0.0 which is incompatible.


# Induction


In [ ]:
import knuckledragger as kd

def Nat(x):
    x = Int(x)
    x.wf = x >= 0
    return x
def Nats(xs):
    return [Nat(x) for x in xs.split()]

#
def RInt(x):
    x = Real(x)
    x.wf = ToReal(ToInt(x)) == x
    return x

def RNat(x):
    x = Real(x)
    x.wf = x >= 0 & ToReal(x) == x
    return x



def pos_induct(P):
    x = FreshInt("x")
    return kd.axiom(
        Implies(And(kd.QForAll([x], x <= 0, P(x)), kd.QForAll([x],P(x),P(x+1))), 
        ForAll([x], P(x))))


## Floor Ceil
https://en.wikipedia.org/wiki/Floor_and_ceiling_functions
https://stackoverflow.com/questions/42133675/get-fractional-part-of-real-in-qf-ufnra/42147549#42147549

A surprising pain in the ass.

floor is the least integer as a definition?


In [58]:
import knuckledragger as kd
from z3 import *
x,y = Reals("x y")
floor = kd.define("floor", [x], z3.ToReal(z3.ToInt(x)))
#floor_idem = kd.lemma(ForAll([x], floor(floor(x)) <= floor(x)), by=[floor.defn], timeout=5000)
#floor_idem = kd.lemma(ForAll([x], floor(floor(x)) == floor(x)), by=[floor.defn, floor_idem], timeout=5000)
n, m = Ints("n m")
int_real_galois = kd.lemma(ForAll([x,n], (x < ToReal(n)) == (ToInt(x) < n)))
int_real_galois_ge = kd.lemma(ForAll([x,n], (ToReal(n) <= x) == (n <= ToInt(x))))
#_1 = kd.lemma(ForAll([n],z3.ToInt(z3.ToReal(n)) == n))
_2 = kd.lemma(ForAll([x], z3.ToInt(floor(x)) == z3.ToInt(x)), by=[floor.defn])

floor_idem = kd.lemma(ForAll([x], floor(floor(x)) == floor(x)), by=[floor.defn, _2]) # nice. got it. Needed lemma _2
floor_le = kd.lemma(ForAll([x], floor(x) <= x), by=[floor.defn])
floor_gt = kd.lemma(ForAll([x], x < floor(x) + 1), by=[floor.defn])



#floor_mono = kd.lemma(kd.QForAll([x,y], x <= y, floor(x) <= floor(y)), by=[floor.defn, floor_idem])
toreal_mono = kd.lemma(ForAll([n,m], (n <= m) == (z3.ToReal(n) <= z3.ToReal(m))))


#floor_minint = kd.lemma(ForAll([x, n], Implies(ToReal(n) <= floor(x), ToReal(n) <= x)), by=[floor.defn])
# maybe there is some wya to get this one?
# should follow from falois_get + monotonicit if ToReal
#floor_minint2 = kd.axiom(ForAll([x, n], Implies(ToReal(n) <= x, ToReal(n) <= floor(x))), by=[floor_minint])
#floor_minint = kd.lemma(ForAll([x, n], (ToReal(n) <= floor(x)) == (ToReal(n) <= x)), by=[floor_minint2, floor_le])
c = kd.Calc([n,x], z3.ToReal(n) <= x)
c.eq(n <= z3.ToInt(x))
#c.eq(z3.ToReal(n) <= z3.ToReal(z3.ToInt(x))) # monotonicty of ToReal. Doesn't need the hint
c.eq(z3.ToReal(n) <= floor(x), by=[floor.defn])
floor_minint = c.qed()


"""
ceil = kd.define("ceil", [x], floor(x) + 1) 
# actually this is probably wrong. Nice. See this is exactly why knuckledragger is kind of nice.
# a counterexample wou'dve been nice
floor_plus_n = kd.lemma(ForAll([x,n], floor(x + ToReal(n)) == floor(x) + ToReal(n)), by=[floor.defn])
floor_plus_1 = kd.lemma(ForAll([x], floor(x+1) == floor(x) + 1), by=[floor.defn])
floor_1 = kd.lemma(floor(1) == 1, by=[floor.defn])
c = kd.Calc([x], ceil(ceil(x)))
c.eq(floor(floor(x) + 1) + 1, by=[ceil.defn])
c.eq()
#ceil_idem = kd.lemma(ForAll([x], ceil(ceil(x)) == ceil(x)), by=[ceil.defn, floor_idem, floor_plus_1, floor_1])
"""

ceil = kd.define("ceil", [x], smt.If(floor(x) == x, floor(x), floor(x) + 1))

ceil_floor = kd.lemma(ForAll([x], floor(x) <= ceil(x)), by=[ceil.defn])

ceil_idem = kd.lemma(ceil(ceil(x)) == ceil(x), by=[ceil.defn, floor.defn])
# hmm. so maybe the way to nitpick is to peel off the outer forall.


LemmaError: (ceil(ceil(x)) == ceil(x), 'Countermodel', [knuckledragger_goal = True,
 x = 0,
 by_1 = True,
 by_0 = True,
 ceil = [0 -> 1, 1 -> 2, else -> 1 + floor(Var(0))],
 floor = [0 -> 0, 1 -> 1, else -> ToReal(ToInt(Var(0)))]])

In [42]:
prove(ToReal(ToInt(ToReal(ToInt(RealVal(3.8))))) == ToReal(ToInt(RealVal(3.4))))

proved


# Pow

https://en.m.wikipedia.org/wiki/Tarski's_high_school_algebra_problem
https://en.m.wikipedia.org/wiki/Exponential_field

https://en.m.wikipedia.org/wiki/Exponentially_closed_field

https://en.m.wikipedia.org/wiki/Schanuel%27s_conjecture
https://en.m.wikipedia.org/wiki/Richardson%27s_theorem - 


The sum formula for expoentnation and hence log / sine seem to be nearly algebraic definitions
https://math.stackexchange.com/questions/702414/what-is-exponentiation
https://math.stackexchange.com/questions/3336/how-many-ways-are-there-to-define-sine-and-cosine
https://math.stackexchange.com/questions/63102/how-to-prove-periodicity-of-sinx-or-cosx-starting-from-the-taylor-seri

Rationalk exponentiation relation
define("ratexp", [x,y,n,m], pow(x,n) == pow(y,m))
+ something that says exp is continuous.

pow(x,y) * pow(x,z) == pow(x, y + z)
pow(pow(x,y), z) == pow(x, y * z)



Continuous recursion. How do we dignify our definitions?
https://www.sciencedirect.com/science/article/pii/0304397595002480
https://www.sciencedirect.com/science/article/pii/S0885064X16300255

y(0) = 

this defines ln by quadrature
t * y' = 1 
Then exp is inverse/.

https://math.stackexchange.com/questions/4202/induction-on-real-numbers
https://arxiv.org/abs/1811.00284 First-Order Continuous Induction, and a Logical Study of Real Closed Fields

https://en.m.wikipedia.org/wiki/Hahn_series
https://en.m.wikipedia.org/wiki/Transseries https://arxiv.org/abs/0801.4877 transseries for beginners
https://en.m.wikipedia.org/wiki/Formal_power_series

In [8]:
import knuckledragger as kd
from z3 import *
x = Real("x")
pow2 = Function("pow2", RealSort(), RealSort())
pow2_mul = kd.axiom( ForAll([x], pow2(x) * pow2(y) == pow2(x + y)))
pow2_one = kd.axiom(pow2(1) == 2)
pow2_zero = kd.axiom(pow2(0) == 1)
pow2_pow2 = kd.axiom(ForAll([x], pow2(pow2(x)) == pow2(2*x)))




In [9]:
import knuckledragger as kd
from z3 import *

x = Real("x")
n, m = Reals("n m")
pow2 = kd.define("pow2", [x], 2 ** x)
pow2_add_raw = kd.axiom(ForAll([n,m], (2**n) * (2 ** m)  == 2**(n + m)))
pow2_add = kd.lemma(ForAll([n,m], pow2(n) * pow2(m) == pow2(n + m)), by=[pow2.defn, pow2_add_raw])
pow2_1 = kd.lemma(ForAll([n], pow2(1) == 2), by=[pow2.defn])
# kd.lemma(ForAll([x,n,m], (x**n) * (x ** m)  == x**(n + m)))
#kd.lemma(ForAll([n,m], (2.1**n) * (2.1 ** m)  == 2.1**(n + m)))
kd.lemma(ForAll([x], x**2 * x**3 == x ** 5))
kd.lemma(ForAll([n,m], (2**(n + 1)) * (2 ** m)  == 2**(n + m + 1)), by=[pow2_add_raw])
#pow2_one = kd.lemma(ForAll([n], 2 ** (n + 1) == 2 * (2 ** n)), by=[pow2_add])
c = kd.Calc([n], 2 * 2 ** n)
c.eq(RealVal(2) ** 1 * 2 ** n )
c.eq(pow2(1) * pow2(n) , by=[pow2.defn])
c.qed()
c.eq(pow2(n + 1), by=[pow2_add])
#c.qed() # fails
c.eq(2 ** (n + 1), by=[pow2.defn])
c.lemmas
# c.qed() #fails. Yikes.

c = kd.Calc([n], 
     pow2(n + 1) - pow2(n))
c.eq(pow2(1) * pow2(n) - pow2(n), by=[pow2_add])
c.eq(2 * pow2(n) - pow2(n),       by=[pow2_1])
c.eq(pow2(n))
pow2_diff = c.qed()

# solve difference equation by induction

# And(P(0), ForAll([n], P(n) == P(n+1))) == ForAll([n], P(n))

#kd.lemma(pow2(n + 1) - pow2(n) == pow2(n), by=[pow2.defn])


In [89]:
import knuckledragger as kd
from knuckledragger.theories.Real import R
from knuckledragger.utils import calc
from z3 import *
x,y,z = Reals('x y z')
pow = Function("pow", R, R, R)
# I'm not sure pow should even be define for x < 0. Is this a bug?
pow_add = kd.axiom(ForAll([x, y, z], pow(x, y + z) == pow(x, y) * pow(x, z)))
pow_
# not needed as axiom? But needs some kind of funky inductionlike thing to generally expression real mul in terms of real add
pow_mul = kd.axiom(ForAll([x, y, z], pow(x, y * z) == pow(pow(x, y), z)))

pow_pos = kd.axiom(ForAll([x,y], Implies(x >= 0, pow(x, y) >= 0)))

mul_pow = kd.axiom(ForAll([x, y, z], pow(x * y, z) == pow(x, z) * pow(y, z)))
pow_zero = kd.axiom(ForAll([x, y], pow(x, 0) == 1))
pow_one = kd.axiom(ForAll([x], pow(x, 1) == x))
_1 = kd.lemma(ForAll([x], pow(x, 1) * pow(x, 1) == x * x), by=[pow_one])
_2 = kd.lemma(ForAll([x], pow(x, 2) == pow(x,1) * pow(x,1)), by=[pow_add])
pow_two = kd.lemma(ForAll([x], pow(x, 2) == x * x), by=[_1, _2])

pow_two = calc(
    pow(x,2),            pow_add,
    pow(x,1) * pow(x,1), pow_one,
    x * x,
    vars=x,
    #by=[pow_add, pow_one]
    )

pow_three = calc(
    pow(x,3),            pow_add,
    pow(x,2) * pow(x,1), pow_one, pow_two,
    x * x * x,
    vars=x,
    )



Pow eval tactic
For rational values of powers, we can build an exact polynomial that says what the solution is.
Kind of part of a general simp tactic?


In [146]:
c = Calc([x,y], pow(x,y))
c.eq(pow(x,1) * pow(x,y-1), by=[pow_add])
c.eq(x * pow(x,y-1), by=[pow_one])
pow_pred = c.qed()
#kd.lemma(ForAll([x, y], pow(x, y) == x * pow(x, y - 1)), by=[pow_add, pow_one])



In [149]:
pow_rec = kd.lemma(ForAll([x, y], pow(x, y) == If(y == 0, 1, x * pow(x, y - 1))), by=[pow_pred, pow_zero])

In [45]:
pow_mul_comm = kd.lemma(ForAll([x, y, z], pow(pow(x, z),y) == pow(pow(x, y), z)), by=[pow_mul])

In [137]:
_1 = calc(
    pow(x,-1) * pow(x,1),   pow_add,
    pow(x,0),               pow_zero,
    1,
    vars=x,
    )

pow_none = kd.lemma(ForAll([x], pow(x, -1) == 1/x), by=[_1, pow_one])


#pow_neg = #kd.lemma(ForAll([x, y], pow(x, -y) == 1/pow(x, y)), by=[pow_none, pow_mul])
pow_neg_cancel = kd.lemma(ForAll([x,y], pow(x,y) * pow(x,-y) == 1), by=[pow_zero, pow_add])
pow_neg = kd.lemma(ForAll([x,y], pow(x,-y) == 1/pow(x,y)), by=[pow_neg_cancel])

In [ ]:
#ipshitz = kd.define("liphitz", [f,C], ForAll([x], f() <= C[x]) )
#diff = kd.define("diff", [f,g], ForAll([x,dx], f(x+dx(eps)) - f(x) <= g(x) * dx(eps)) 

In [91]:
pow_sqrt = calc(
    pow(x, 1/2) * pow(x, 1/2),            pow_add,
    pow(x,1),                             pow_one,
    x,
    vars=x,
)
#kd.lemma(ForAll([x], pow(x, 1/2) * pow(x, 1/2) == x), by=[pow_mul])

#kd.lemma(And(1.4 <= pow(2,1/2), pow(2,1/2) <= 1.42), by=[pow_sqrt])
kd.lemma(pow(2,1/2) <= 1.42, by=[pow_sqrt])
# Hmm. Can I prove pow is positive? I'm not sure.
kd.lemma(-1.42 <= pow(2,1/2), by=[pow_sqrt])
kd.lemma(1.4 < pow(2,1/2), by=[pow_sqrt, pow_pos])


|- pow(2, 1/2) > 7/5

In [61]:
import numpy as np
np.sqrt(2)

1.4142135623730951

In [135]:
def ForAll(vars, expr):
    if len(vars) == 0:
        return expr
    else:
        return z3.ForAll(vars,expr)

# Hoh yeah, I should replace calc. This is way better

#https://note.nkmk.me/en/python-method-chain-line-break/

#class Lemma(): .have().

In [ ]:
# old calc
def calc(*args, vars=None, by=[], op=eq) -> Proof:
    """
    calc is for equational reasoning.
    One can write a sequence of formulas interspersed with useful lemmas.
    Inequational chaining can be done via op=lambda x,y: x <= y
    """

    def thm(lhs, rhs):
        if vars == None:
            return op(lhs, rhs)
        else:
            return z3.ForAll(vars, op(lhs, rhs))

    lemmas = []
    local_by = []
    lhs = args[0]
    for arg in args[1:]:
        if is_proof(arg):
            local_by.append(arg)
        else:
            lemmas.append(lemma(thm(lhs, arg), by=by + local_by))
            lhs = arg
            local_by = []
    return lemma(thm(args[0], args[-1]), by=by + lemmas)


In [81]:

# Yeah, I can prove pow pow examples
pow_4 = (
    Calc([x], pow(pow(x, 2), 2))
    .eq(pow(pow(x, 2),1) * pow(pow(x, 2), 1), by=[pow_add])
    .eq(pow(x, 2) * pow(x, 2),                by=[pow_one])
    .eq(pow(x, 4),                            by=[pow_add])
    .qed()
)

In [86]:
c = Calc([x], pow(pow(x, 2), 2))
c.eq(pow(pow(x, 2),1) * pow(pow(x, 2), 1), by=[pow_add])
c.eq(pow(x, 2) * pow(x, 2),                by=[pow_one])
c.eq(pow(x, 4),                            by=[pow_add])
pow_4 = c.qed()

|- ForAll(x, pow(pow(x, 2), 2) == pow(x, 4))

# Exp

In [16]:
import knuckledragger as kd
import z3
exp = z3.Function("exp", kd.R, kd.R)

x,y,z = z3.Reals('x y z')
#exp_zero = kd.axiom(exp(0) == 1)
exp_add = kd.axiom(z3.ForAll([x, y], exp(x + y) == exp(x) * exp(y)))
exp_lower = kd.axiom(z3.ForAll([x], exp(x) >= 1 + x)) # tight low approximation at x = 0.

exp_pos = kd.axiom(z3.ForAll([x], exp(x) > 0)) # maybe we can derrive this one.
exp_zero = kd.lemma(z3.ForAll([x], exp(0) == 1), by=[exp_add, exp_pos])

# exp(0) = exp(0 + 0) = exp(0) * exp(0) ===> 1 = 1 * 1 or 0 = 0 * 0
c = kd.Calc([], exp(0))
c.eq(exp(0 + 0))
c.eq(exp(0) * exp(0), by=[exp_add])
_1 = kd.lemma(z3.Or(exp(0) == 0, exp(0) == 1), by=[c.qed()])
exp_zero = kd.lemma(exp(0) == 1, by=[_1, exp_pos])

exp_div = kd.lemma(z3.ForAll([x, y], exp(x)*exp(-x) == 1), by=[exp_add, exp_zero])
exp_inv = kd.lemma(z3.ForAll([x], exp(-x) == 1/exp(x)), by=[exp_div])
e = kd.define("e", [], exp(1))


exp_inv_lower = kd.lemma(z3.ForAll([x], exp(-x) >= 1 - x), by=[exp_lower])

exp_upper = kd.lemma(z3.ForAll([x], exp(-x) * exp(x) >= (1-x) * exp(x)), by=[exp_inv_lower, exp_pos])
exp_upper = kd.lemma(z3.ForAll([x], 1 >= (1 -x) * exp(x)), by=[exp_upper, exp_div])
exp_upper = kd.lemma(kd.QForAll([x], x < 1, exp(x) <= 1/(1 - x)), by=[exp_upper])


c = kd.Calc([x], 1)
c.eq(exp(x) * exp(-x), by=[exp_div])
c.ge((1 - x) * exp(x), by=[exp_inv_lower, exp_pos])
c.qed()


#kd.lemma(e() >= 2, by=[e.defn, exp_lower])

#c = kd.Calc([x], exp(x))
#c.eq(exp(x/2) * exp(x/2), by=[exp_add])
#_1 = c.qed()

#kd.lemma(e() >= 2.5, by=[e.defn, exp_lower, _1])
#linapprox = exp(x) == exp(x + a - a) == exp(a)*exp(x - a) >= exp(a)*(1 + x - a)



LemmaError: ('lemma', ForAll(x, exp(x) > 0), unknown)

In [ ]:
#exp_pos = kd.lemma(z3.ForAll([x], exp(x) > 0), by=[exp_lower, exp_add]) # maybe we can derrive this one.
# streatgey = reduced exp(x) ==>  y = n*x st  -1 < y, and then use taylor lower approx
c = Calc([x], exp(x))
#c.eq(exp(), by=[exp_add])

In [51]:
from z3 import *
x,y,z = Reals('x y z')
prove(Implies(And(y == 1/3, x > 1), (x**y) > 0))

failed to prove
[y = 3333333333333333/10000000000000000]


In [57]:
prove(ForAll([x], Implies(x >= 0, (x**.5) **2 == x)))

failed to prove


Z3Exception: model is not available

In [129]:
#def exists_schema( name, args, body):

sqrt = kd.define("sqrt", [x], pow(x, 1/2))
sqrt_square = kd.lemma(ForAll([x], sqrt(x) * sqrt(x) == x), by=[pow_sqrt, sqrt.defn])


euler = Const("e", R)
euler_approx = kd.axiom(2.7 < euler, euler < 2.8)
exp = kd.define("exp", [x], pow(euler, x))
exp_approx = kd.axiom(ForAll([x], exp(x) >= 1 + x))

nexp_approx = kd.lemma(ForAll([x], exp(-x) >= 1 - x), by=[exp_approx])

c = Calc([], exp(-1))
c.eq(pow(euler, -1), by=[exp.defn])
c.eq(1/euler,        by=[pow_none])
c.eq(1/pow(euler,1), by=[pow_one])
c.eq(1/exp(1),       by=[exp.defn])
exp1_inv = c.qed()
#kd.lemma(ForAll([x], exp(-1) == pow(euler,-1)), by=[exp.defn, pow_none, euler_approx])



In [ ]:
import knuckledragger as kd
import knuckledragger.theories.Real as R

def exp_bound_n(m):
    kd.lemma(exp(x / 2**m) >= 1 + x / 2**m, by=[exp_lower])
    acc = 2**m
    for i in range(m):
        acc = acc / 2
        kd.lemma(exp(x / acc) >= 1 + x / acc, by=[exp_lower])



_1 = kd.lemma(exp(x / 4) * exp(x / 4) == exp(x / 2), by=[exp_add])
_2 = kd.lemma(exp(x / 2) * exp(x / 2) == exp(x), by=[exp_add])
_3 = kd.lemma(exp(x / 4) >= 1 + x / 4, by=[exp_lower])
_4 = kd.lemma(exp(x / 4)**4 == exp(x), by=[_1, _2])
(1 + x/4)**4 >= exp(x/4)**4 





# Polynomials

Lagrange interpolation
falling factorials
finite difference
polynomial rolle

Maybe don't try to encode a general concept at first.



diff := f(x) = f(a) + something*(x - a)

(f(x) - f(a)) / (x - a) _does not work_, because there is a divide by zero.
The 

p(x) = p(0) + (p(x) - p(0) / x)*x

p(x) - p(0) / x is true in a sense. It has to be. But not in the real function sense. Polynomial division.

exp(x) = exp(a) + (exp(x) - exp(a) / (x - a)) * (x - a)
but
1 + x + x**2/2 - 1 + a + x**2
exp(x) = e(a) + exp(a)(x - a) + exp(a)(x-a)**2/2 + ...
exp(x) = exp(a) +  (exp(a) * (exp(x))  ) * (x - a)

exp(x) - exp(a) / (x - a) = exp(y)
exp(x) - exp(a) = exp(y) * (x - a) # no pole now.

exp(x) = exp(a) + exp(y(x,a)) * (x - a)
min(x,a) <= y(x,a) <= max(x,a)

singularity removal. Only the right choice keeps the remainder function continuous
y = If(x == a , exp(a), ln(exp(x) - exp(a)) - ln(x - a))


exp(x) = exp(a) + exp(y)*(x-a)


p(x) = p(0) + p'(eps(x))*x
This is the lagrange form of the remainder based on intermeidtae value theorem.

exact finite integration
 = p'(eps)(b - a) + p'(eps2)(c - b) + ...



 singularity removal using sympy limit features.

In [ ]:
horner 

# Ln

$\int 1/t = ln(t) $

Mean value theorem? Extreme value theorem? Intermediate?




In [ ]:
ln = z3.Function("ln", RealSort(), RealSort())
ln_def = kd.axiom(ForAll([x], ln(exp(x)) == x))
# is this sufficient?

# Geometric


In [19]:
def geom(n):
    x = z3.Real("x")
    return kd.lemma(z3.ForAll([x], (1 - x) * (1 + z3.Sum([x**i for i in range(1,n)]) )== 1 - x**n))
geom(2)





#z3.prove((1 - x)*(1 + x**1) == 1 - x**2)

|- ForAll(x, (1 - x)*(1 + x**1) == 1 - x**2)

In [ ]:
# singularity removal
# singularty removal using sympy to do limits in closed form?
# take closed form as defition. Summation formula as property.
kd.define("geom", [x,n], smt.If(x == 1, n, (1 - x**n) / (1 - x)))
kd.lemma(ForAll([x,n], geom(x,n) == x**n + geom(x, n + 1)))

In [7]:
# maybe the generalized notion of sum is too much abstraction
import knuckledragger as kd
import z3
n = z3.Int("n")
x = z3.Real("x")
geom = kd.kernel.define_fix("geom", [x, n], kd.R, lambda geom: z3.If(n <= 0, 1, x**n + geom(x, n - 1)))
pow_ax = kd.axiom(z3.ForAll([x,n], x*x**n == x**(n+1)))
c = kd.Calc([x,n], x * geom(x,n), assume=[n > 0])
c.eq(x*(x**n + geom(x,n-1)), by=[geom.defn])
c.eq(x**(n+1) + x * geom(x,n-1), by=[pow_ax])
#c.eq(geom(x,n+1), by=[geom.defn])



... == x**ToReal(n + 1) + x*geom(x, n - 1)

# Binomial

(1 + x)^n
https://en.wikipedia.org/wiki/Binomial_theorem
https://en.wikipedia.org/wiki/Binomial_series
https://en.wikipedia.org/wiki/Falling_and_rising_factorials



In [ ]:
n,m = Int("n m")
x = Real("x")
fact = kd.define("fact", [n], If(n <= 0, 1, n * fact(n-1)))
fallfact = kd.define("fallfact", [x,m], If(m <= 0, 1, x * fallfact(x-1,m-1)))
binom

pow(x + y, n + 1) ==  x * pow(x + y, n) + y * pow(x + y, n)




# Area




In [ ]:
MeasureSet = Record("MSet" Set(), pred=lambda x: measurable(x))
OpenSet =  Record("Open", )
Closed = 


def CSeq(T): # countable sequence
    return ArraySort(IntSort(), T)
bigunion = Function("bigunion", Set(Set(R)), R)
countinter = Function("countinter", ArraySort(IntSort(), Set(R)), Set(R))
Implies(ForAll([x], Implies(s[x] , open(x))), open(bigunion(s)))
runion = Function("runion", ArraySort(R, Set(R)), Set(R)) 
# runion(f) == bigunion(Lambda([s], Exists([x], s[x] == f[x]))



Just axiomatize the area of triangles.

In [ ]:
tri = kd.define()
rect = SetUnion(tri(a,b,c), tri(d,b,c))
area(tri(pq,q,r)) = 1/2 * abs((p - q).x * (p - r).y -  )

area()

def triangulate(S):

def outer_inner(S,N):
    Minimize()


In [ ]:
S2 = SetSort(V2)
A,B,C = Consts("A B C", S2)

kd.notation.le.define([A,B], IsSubset(A,B))
kd.notation.add.define([A,B], Union(A,B))
kd.notation.sub.define([A,B], Difference(A,B))
kd.notation.mul.define([A,B], Intersection(A,B))

Function("area", S2, RealSort())
p = Const("p", V2)

l,r,u,d = Reals("l r u d")
area_subset = kd.axiom(kd.QForAll([A,B], A <= B, area(A) <= area(B)))
rect = kd.define("rect", [l,r,u,d], Lambda([p],  And(l <= p.x, p.x <= r, u <= p.y, p.y <= d)))
area_rect = kd.axiom(kd.ForAll([l,r,u,d], area(rect(l,r,u,d)) == If(And(r >= l, u >= d), (r - l) * (d - u), 0)))

area_nonneg = kd.lemma(kd.QForAll([A], area(A) >= 0))
area_union = kd.lemma(kd.ForAll([A,B], area(A + B) == area(A) + area(B) - area(A*B)), by=[area_subset])

# area translate invartiant
# area rotate
# area affine

circle = kd.define("circle", [p, r], Lambda([q], (p.x - q.x)**2 + (p.y - q.y)**2 <= r**2))

circle_outer = kd.lemma(kd.ForAll([p,r], circle(V0,1) <= rect(-1,1,-1,1)))
circle_inner = kd.lemma(kd.ForAll([x], And(x**2 = 2, rect(-x,x,-x,x) <= circle(V0,1))))


#area_parallogram = 
# area_righttriangle
#area_triangle = 
translate = kd.define("translate", [p, A], Lambda([q], A(q - p)))
kd.axiom(kd.ForAll([p,A], area(translate(p,A)) == area(A))




# Complex Exp


In [ ]:
exp = z3.Function("exp", kd.Complex, kd.Complex)

# Trig
Fu, Hongguang, Xiuqin Zhong, and Zhenbing Zeng. “Automated and readable simplification of trigonometric expressions.” Mathematical and computer modelling 44.11 (2006): 1169-1177. https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.657.2478&rep=rep1&type=pdf
https://docs.sympy.org/dev/modules/simplify/fu.html

Chebyshev polynomials basically give a formula for the rational multiples of pi.
Don't persay need to mention


rational trignometry. https://web.maths.unsw.edu.au/~norman/Rational1.htm
https://www.cut-the-knot.org/pythagoras/RationalTrig/CutTheKnot.shtml
"angles don't exist" Only the combo sin(theta) not raw theta.


https://en.wikipedia.org/wiki/List_of_trigonometric_identities


Flint has arb and calcium now folded in
<https://fredrikj.net/python-flint/>



Use arb as knuckledragger style axiom schema.

What about user defined propagator in z3?

```python
def cos_schema(x):
    y = flint.cos(x)
    z = FreshReal()
    return ForAll([z], Implies(And(0 <= x. x <= ), And(y.lo <= cos(z), cos(z) <= y.hi)))

def flint_to_interval(i):
    return Interval.mk(RealVal(i.lo), RealVal(i.hi))

def cos_schema(i : flint.interval) -> Proof:
    j = Const("j", Interval) #we don't need fresh here.
    return axiom(QForAll([j], j <= i,  cos(j) <= flint.cos(j)))


```

The p(x) * f(x) >= q(x) is a nice way of getting an non conditional constraint. We can do it even for exp.
 https://en.wikipedia.org/wiki/Puiseux_series
 https://en.wikipedia.org/wiki/Pad%C3%A9_approximant

 For differential equations:

d(p(x) * y(x)) >= d(q(x))
sum of squares something. The p gives you a way of giving a region. This was akin to the S technique.
p y' >= q 

These "nice" odes probably have closed hypergeomtric solutions or something.
Maybe there's a general definitional property of hypergeometric.


These mnatching on * - + are not workign frustratingly.
1. use sympy / flint to validate
2. use the hidden definitions plus,mul

In [6]:
import knuckledragger as kd
import knuckledragger.theories.Real as R
from z3 import *
cos = Function("cos", kd.R, kd.R)
sin = Function("sin", kd.R, kd.R)
x,y,z = Reals("x y z")
cos_lower = kd.axiom(ForAll([x],cos(x) >= 1 - x**2))
sin_approx = kd.axiom(ForAll([x], x * sin(x) <= x**2))

sin_asym = kd.axiom(ForAll([x], sin(-x) == -sin(x)))
sin_zero = kd.lemma(ForAll([x], sin(0) == 0), by=[sin_asym])
sin_sub_x = kd.lemma(kd.QForAll([x], x >= 0, sin(x) <= x ), by=[sin_approx, sin_zero])
#sin_zero = kd.axiom(sin(0) == 0)

cos_sym = kd.axiom(ForAll([x], cos(-x) == cos(x)))


cos_zero = kd.axiom(cos(0) == 1)

sin_sum = kd.axiom(ForAll([x,y], sin(x+y) == sin(x) * cos(y) + cos(x) * sin(y)))
cos_sum = kd.axiom(ForAll([x,y], cos(x+y) == cos(x) * cos(y) - sin(x) * sin(y)))

#sin_cos_2 = kd.lemma(ForAll([x], sin(x)**2 + cos(x)**2 == cos(x) * cos(-x) - sin(-x) * sin(x)), by=[cos_sym, sin_asym])
c = kd.Calc([x], sin(x)**2 + cos(x)**2)
c.eq(cos(x) * cos(-x) - sin(-x) * sin(x), by=[cos_sym, sin_asym])
c.eq(cos(x - x), by=[cos_sum])
c.eq(1, by=[cos_zero])
c.qed()
#kd.lemma(ForAll([x], cos(x + y) == R.plus(R.mul(cos(x), cos(y)), R.mul(sin(x), sin(y)))), by=[cos_sum, R.mul.defn, R.plus.defn])

#c.eq(cos(x - x), by=[cos_sum])

#cos_symm = kd.lemma(ForAll([x], cos(x) == cos(-x)), by=[cos_sum, sin_sum, sin_zero, cos_zero])




|- ForAll(x, sin(x)**2 + cos(x)**2 == 1)

In [ ]:
import knuckledragger.theories.Complex as C
cexp = Function("cexp", R, Complex)
kd.axiom(ForAll([x,y], cexp(x + y) == cexp(x) * cexp(y)))
kd.axiom(cexp(0) == C1)
pi = Real("pi")
kd.axiom(cexp(pi), -C1)
sin = kd.define([t], cexp(t).im)
cos = kd.define([t], cexp(t).re)

sin_sum = kd.lemma()
cos_sum = kd.lemma()


In [ ]:
"""
c = kd.Calc([x], sin(x))
c.eq(sin(0 + x))
c.eq(sin(0) * cos(x) + cos(0) * sin(x), by=[sin_sum])
c.eq(0 * cos(x) + 1 * sin(x), by=[sin_zero, cos_zero])
"""
"""
c = kd.Calc([x], sin(x - x))
c.eq(sin(0))
c.eq(0, by=[sin_zero])
c.eq(sin(x) * cos(-x) + cos(x) * sin(-x), by=[sin_sum])
c.eq()


c = kd.Calc([x], cos(x-x))
c.eq(cos(0))
c.eq(1, by=[cos_zero])
c.eq(cos(x) * cos(-x) - sin(x) * sin(-x), by=[cos_sum])
"""


In [ ]:

import flint
x = flint.arb(mid=7, rad=1e-10)
#dir(x)
flint.arb_poly([x, 1]).integral().derivative()
flint.arb_poly.interpolate([0, 1], [0, 1])



In [111]:
pimid = RealVal(flint.arb.pi().mid().str(5, radius=False))
pirad = RealVal(flint.arb.pi().rad().str(5, radius=False))
pirad
pimid

3927/1250

In [113]:
type(flint.arb.pi())

flint.types.arb.arb

In [115]:
def z3_of_arb(x : flint.types.arb.arb) -> tuple[ExprRef, ExprRef]:
    mid = RealVal(x.mid().str(5, radius=False))
    rad = RealVal(x.rad().str(5, radius=False))
    return mid, rad

In [105]:
flint.arb.pi().upper()

[3.14159265358979 +/- 3.57e-15]

In [104]:
flint.arb.pi().lower().str(5, radius=False)

'3.1416'

In [112]:
pi = Real("pi")
import flint
pi_bnd = kd.axiom(And(pimid - pirad < pi, pi < pimid + pirad))
pi_bnd

|- And(3927/1250 - 5551/25000000000000000000 < pi,
    pi < 3927/1250 + 5551/25000000000000000000)

In [134]:
def sin_bnd(x):
    x = flint.arb(x).sin()
    mid = RealVal(x.mid().str(5, radius=False))
    rad = RealVal(x.rad().str(5, radius=False))
    return mid, rad

def flint_schema(z3_fun, flint_fun):
    def res(x):
        x = flint_fun(flint.arb(x))
        mid = RealVal(x.mid().str(5, radius=False))
        rad = RealVal(x.rad().str(5, radius=False))
        kd.axiom(And(mid - rad < z3_fun(RealVal(x)), z3_fun(RealVal(x)) < mid + rad))
    return res

cos_bnd = flint_schema(cos, flint.arb.cos)
cos_bnd(1)

Z3Exception: b'parser error'

In [117]:
# just bolting in the initial taylor series.
sin_approx = kd.axiom(ForAll([x], Implies(x >= 0, sin(x) - x <= 1/RealVal(6) * x**3)))
sin_approx

|- ForAll(x, Implies(x >= 0, sin(x) - x <= (1/6)*x**3))

In [59]:


sin = Function("sin", R, R)
sin_zero = kd.axiom(sin(0) == 0)
sin_90 = kd.axiom(sin(pi/2) == 1)

cos = kd.define("cos", [x], sin(x + pi/2))
cos_zero = kd.lemma(cos(0) == 1, by=[sin_90, cos.defn])
#cos = Function("cos", R, R)
#cos_zero = kd.axiom(cos(0) == 1)
#

#cos_sum = kd.axiom(ForAll([x,y], cos(x + y) == cos(x) * cos(y) - sin(x) * sin(y)))

#cos_sin_sq = kd.axiom(ForAll([x], cos(x)**2 + sin(x)**2 == 1))

#cos_doub = kd.lemma(ForAll([x], cos(2*x) == cos(x)**2 - sin(x)**2), by=[cos_sum])



NameError: name 'R' is not defined

# Quaternion
Quaternions so we cna try the frama-C stuff.


In [20]:
import knuckledragger as kd
from z3 import *
Q = kd.Record("Quaternion", ("re", kd.R), ("i", kd.R), ("j", kd.R), ("k", kd.R))
x,y,z = Consts("x y z", Q)
kd.notation.add.define([x,y], Q.mk(x.re + y.re, x.i + y.i, x.j + y.j, x.k + y.k))
kd.notation.mul.define([x,y], Q.mk(x.re * y.re - x.i * y.i - x.j * y.j - x.k * y.k, 
                                   x.re * y.i + x.i * y.re + x.j * y.k - x.k * y.j, 
                                   x.re * y.j - x.i * y.k + x.j * y.re + x.k * y.i, 
                                   x.re * y.k + x.i * y.j - x.j * y.i + x.k * y.re))
conj = kd.define("conj", [x], Q.mk(x.re, -x.i, -x.j, -x.k))
norm2 = kd.define("norm2", [x], x.re**2 + x.i**2 + x.j**2 + x.k**2)
norm2_pos = kd.lemma(ForAll([x], norm2(x) >= 0), by=norm2.defn)





TypeError: 'Proof' object is not iterable

# Convex
Ectended reals and convex analysis

https://francisbach.com/jensen-inequality/

boyd



In [16]:
from z3 import *
a,b,c,x = Reals("a b c x")
f = Lambda([x], a * x**2 + b * x + c)
prove(Implies(a != 0, a * x**2 + b * x + c == a * (x + b / 2 / a)**2 + c - b**2 / 4 / a))
prove(Implies(a > 0, a * (x + b / 2 / a)**2 + c - b**2 / 4 / a >=  c - b**2 / 4 / a))
prove(Implies(a > 0, a * x**2 + b * x + c >=  c - b**2 / 4 / a))

#argmin = Function("argmin", ArraySort(RealSort(), RealSort()), RealSort())

proved
proved
proved


In [9]:
e = a * (x + b / 2 / a)**2 + c - a * b**2 / 4
simplify(e)

a*(x + (1/2*b)/a)**2 + c + -1/4*a*b**2

# Geometry

Ok, maybe the fully symbolic method is idiotic. What about just coordinatizing things


In [10]:
import knuckledragger as kd
import z3
Point = kd.Record("Point", ("x", kd.R), ("y", kd.R))
Circle = kd.Record("Circle", ("center", Point), ("radius", kd.R))
p,q,r = z3.Consts("p q r", Point)
c = z3.Const("c", Circle)
on_circle = kd.define("on_circle", [p,c], (p.x - c.center.x)**2 + (p.y - c.center.y)**2 == c.radius**2)

Line = kd.Record("Line", ("a", kd.R), ("b", kd.R), ("c", kd.R))
l,l1,l2= z3.Consts("l l1 l2", Line)
on_line = kd.define("on_line", [p,l], l.a * p.x + l.b * p.y + l.c == 0)

line_equal = kd.define("line_equal", [l1,l2], z3.ForAll([p], on_line(p,l1) == on_line(p, l2)))

dist = kd.define("dist", [p,q], ((p.x - q.x)**2 + (p.y - q.y)**2)**0.5)
quadrance = kd.define("quadrance", [p,q], (p.x - q.x)**2 + (p.y - q.y)**2)
#dist_quad = kd.lemma(z3.ForAll([p,q], dist(p,q)**2 == quadrance(p,q)), by=[dist.defn, quadrance.defn])
kd.utils.simp(dist.defn)

AttributeError: module 'knuckledragger' has no attribute 'utils'

In [85]:
Point = Datatype("Point")
Point.declare("mk_point", ("x", R), ("y", R))
Point = Point.create()

p,q,r = Consts("p q r", Point)

#quadrance

# too fine though. Triangles are an equivalence class of 3 tuples.
Triangle = Datatype("Triangle")
Triangle.declare("mk_triangle", ("p", Point), ("q", Point), ("r", Point))
Triangle = Triangle.create()

tri_eq = kd.define("tri_eq", [t1, t2], Or(
    And(t1.p == t2.p, t1.q == t2.q, t1.r == t2.r),
    And(t1.p == t2.q, t1.q == t2.r, t1.r == t2.p),
    And(t1.p == t2.r, t1.q == t2.p, t1.r == t2.q),
    And(t1.p == t2.r, t1.q == t2.q, t1.r == t2.p),
    And(t1.p == t2.q, t1.q == t2.p, t1.r == t2.r),
    And(t1.p == t2.p, t1.q == t2.r, t1.r == t2.q),
))
import knuckledragger.notation as notation
notation.eq.register(Triangle, tri_eq)

# triangle as set? 3 card set?
#s = Const("s", SetSort(Point))
#istriangle = kd.define("triangleq", [s], SetHasSize(s, 3))

Line = SetSort(Point)
isline = kd.define("isline", [s], ForAll([p,q, x], Implies(s[p] & s[q], s[l*p + (1-l)*q])))
# projective. A point is a scaling closed set
projpoint = kd.define("isprojpoint", [s], ForAll([p, x], Implies(x != 0, s[p] == s[x*p])))




Z3Exception: b'set-has-size is not supported'

# Convergence
https://en.wikipedia.org/wiki/Geometric_series
https://en.wikipedia.org/wiki/Convergence_tests
Alternating series test - gives bounds

https://en.wikipedia.org/wiki/Ratio_test

In [ ]:
from z3 import *
RFun = ArraySort(RealSort(), RealSort())
RSeq = ArraySort(IntSort(), RealSort())


x,y,z,eps = Reals('x y z eps')

f,g,h = Consts('f g h', RFun)

Sum = Function('Sum', RSeq, IntSort(), RealSort())
sum_def = ForAll([n], Sum(a, n) == If(x < 0, 0, a[n] + Sum(a, n-1))) 

def lemma(by=)







So continous convergencew is eaiser really. I don't have to deal with int real interop then?

limsup
liminf
+ extended reals
Now always extists?


In [ ]:
def lim0(f):
    return ForAll([eps], Exists([delta], ForAll([x], And(0 < Abs(x) < delta, Abs(f(x)) < eps))))
def modcont(f, eps):
    return ForAll([x], Abs(f(x)) < eps(x))



In [48]:
def keytest(*args, **kwargs):
    print(args)
    print(kwargs)

keytest(1,2,by=7, 14, x=18)

SyntaxError: positional argument follows keyword argument (3783932439.py, line 5)

In [23]:
import knuckledragger as kd
from z3 import *
import knuckledragger.theories.Real as Real
RSeq = Real.RSeq
n,m,k,N = Ints('n m k N')
a,b = Consts('a b', RSeq)
def induct(P):
    return kd.axiom(Implies(And(P(0), ForAll([n], Implies(And(n >= 0, P(n)), P(n+1)))),
                         ForAll([n], Implies(n >= 0, P(n)))))

induct(lambda n: n >= 0)

sum = Function('sum', Real.RSeq, IntSort(), RealSort())
sum = kd.define("sum", [a,n], If(n < 0, 0, a[n] + sum(a,n-1)))
#sum = kd.define_rec("sum", [a,n], lambda sum: If(n < 0, 0, a[n] + sum(a,n-1)))


def ForAll(vs, body):
    if any(hasattr(v, "wf") for v in vs):
        return z3.ForAll(vs, Implies(And(*[v.wf for v in vs if hasattr(v, "wf")]), body))
    else:
        return z3.ForAll([vs], body)

def Nats(xs):
    xs = Ints(xs)
    for x in xs:
        x.wf = x >= 0
    return xs



kd.lemma(ForAll([n], sum(K(IntSort(), RealVal(0)), n) >= 0), by=[sum.defn, induct(lambda n: sum(K(IntSort(), RealVal(0)), n) >= 0)])






is_nat = kd.define("nat", [x], ToReal(ToInt(x)) == x)

def RInts(xs):
    xs = Reals(xs)
    for x in xs:
        x.wf = is_nat(x) 
    return xs

def RPlus(xs):
    xs = Reals(xs)
    for x in xs:
        x.wf = x >= 0
    return xs

def pos_induct(P):
    return kd.axiom(Implies(And(P(0), ForAll([x],Exists([eps], Implies(And(x >= 0, eps >= 0, P(n)), P(x+eps)))),
                         ForAll([n], Implies(n >= 0, P(n))))

def RNats(xs):
    xs = Reals(xs)
    for x in xs:
        x.wf = And(is_nat(x), x >= 0)
    return xs




       sum(a, n) == If(n < 0, 0, a[n] + Sum(a, n - 1))) to ForAll([a, n],
       sum(a, n) == If(n < 0, 0, a[n] + sum(a, n - 1)))


|- ForAll(n, sum(K(Int, 0), n) >= 0)

# Continuity



In [32]:
import knuckledragger as kd
import knuckledragger.theories.Real as R
import knuckledragger.smt as smt

x,y,z,eps = smt.Reals('x y z eps')
f, delta = smt.Consts("f delta", R.RFun)
unicont = kd.define("unicont", [f,delta], kd.QForAll([x,y,eps], eps > 0, R.abs(x - y) < delta[eps], 
                                                     R.abs(f[x] - f[y]) < eps))
delta = Const("delta", kd.R >> (kd.R >> kd.R))
delta[x]
cont = kd.define("cont", [f,delta], kd.QForAll([x,y,eps], eps > 0, R.abs(x - y) < delta[x][eps], 
                                                               R.abs(f[x] - f[y]) < eps))

# kd.lemma Implies(unicont(f,delta), cont(f,Lambda([x], delta)))
kd.lemma(unicont(Lambda([x], x), Lambda([eps], eps)), by=[unicont.defn])
#kd.lemma(cont(Lambda([x], x*x), Lambda([x], Lambda([eps], 10*x*eps))), by=[cont.defn])

#delta = If(x == 0, )
d = Real("d")
prove(Implies(And(smt.Abs(x - y) < d, eps > 0, eps == 2 * d * smt.Abs(x) + d*d), smt.Abs(x*x - y*y) < eps))


proved


(x**2 + y**2) == 
(x + y)(x - y) == 
abs(x - y)sgn(x - y)(x + y)

## Defining cos
cos(x) = sum(x**n)
https://www.philipzucker.com/analysis_knuckle/

c(0) = 1
c'' = -c 
s(0) = 1
s'' = -s

s^2 + c^2 = 1 is invariant.
s(0)^2 + c(0)^2 = 1
2 s' s + 2 c' c = 0



How to prove periodic? Youch.

Going to summation form
1 - x^2 /2 + ...
alternating series.
not monotonically decreasing until a certain n.




In [3]:
from z3 import *

B = BoolSort()
Z = IntSort()
R = RealSort()
x,y,z = Reals('x y z')

def lemma(thm, by=[], admit=False):
    if not admit:
        #print(Implies(And(by), thm).sexpr())
        prove(Implies(And(by), thm))
    else:
        print("Admitted!!", thm)
    return thm
    
plus = Function("plus", R,R,R)
plus_def = ForAll([x, y], plus(x, y) == x + y)

plus_0 = lemma(ForAll([x], plus(x, 0) == x), by=[plus_def])
plus_comm = lemma(ForAll([x, y], plus(x, y) == plus(y, x)), by=[plus_def])
plus_assoc = lemma(ForAll([x, y, z], plus(x, plus(y, z)) == plus(plus(x, y), z)), by=[plus_def])
                   
mul = Function("mul", R,R,R)
mul_def = ForAll([x, y], mul(x, y) == x * y)

mul_zero = lemma(ForAll([x], mul(x, 0) == 0), by=[mul_def])
mul_1 = lemma(ForAll([x], mul(x, 1) == x), by=[mul_def])
mul_comm = lemma(ForAll([x, y], mul(x, y) == mul(y, x)), by=[mul_def])
mul_assoc = lemma(ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)), by=[mul_def])
mul_distrib = lemma(ForAll([x, y, z], mul(x, plus(y, z)) == plus(mul(x, y), mul(x, z))), by=[mul_def, plus_def])  

Nat = Datatype("Nat")
Nat.declare('zero')
Nat.declare('succ', ('pred', Nat))
Nat = Nat.create()
n,m,k = Consts('n m k', Nat)

def induct(P):
    return Implies(And(P(Nat.zero), ForAll([n], Implies(P(n), P(Nat.succ(n))))),
                   #-----------------------------------------------------------
                   ForAll([n], P(n)))

real = Function("real", Nat, R)
real_def = ForAll([n], real(n) == If(Nat.is_zero(n), 0, plus(1, real(Nat.pred(n)))))

ceil = Function("ceil", R, Nat)
ceil_def = ForAll([x], ceil(x) == If(x <= 0, Nat.zero, Nat.succ(ceil(x-1))))

Seq = ArraySort(Nat, R)
sum_ = Function("sum", Seq, Nat, R)
s = Const('s', Seq)
sum_def = ForAll([s, n], sum_(s, n) == If(n == Nat.zero, 0, plus(s[Nat.pred(n)], sum_(s, Nat.pred(n)))))

abs = Function("abs", R, R)
abs_def = ForAll([x], abs(x) == If(x >= 0, x, -x))

proved
proved
proved
proved
proved
proved
proved
proved


In [ ]:
#https://en.wikipedia.org/wiki/Modulus_of_convergence
Modulus = ArraySort(R, Nat)
N = Const("N", Modulus)
eps = Real("eps")
cauchy = Function(Seq, Modulus, B)
cauchy_def == ForAll([s,N], cauchy(s, N) == ForAll([eps,n,m], Implies(And(n > N(eps), m > N(eps)), 
                                                                       abs(s[n] - s[m]) < eps)))


In [11]:
seq_sum = Function("seq_sum", Seq, Seq, Seq)
s,t = Consts("s t", Seq)
seq_sum_def = ForAll([s, t], seq_sum(s, t) == Lambda([n], plus(s[n], t[n])))

seq_zero = K(Nat, RealVal(0))
seq_zero.sort()
seq_sum_zero = lemma(ForAll([s], seq_sum(s, seq_zero) == s), by=[seq_sum_def, plus_def])




proved


In [ ]:
# Bundling as a record
# This is the reifnment problem.
CSeq = Datatype("CSeq")
CSeq.declare("mk", ("seq", Seq), ("mod", Modulus))
CSeq = CSeq.create()

cauchy = Function("cauchy", CSeq, B)
cauchy_def = ForAll([s], cauchy(s) == cauchy(s.seq, s.mod))


Implies(And(cauchy(M), cauchy(N)))


In [21]:
def myadd(x,y):
    match x.sort():
        case Seq:
            return seq_sum(x,y)
        case _:
            print(s.sort().name())
            assert False, s.sort().name()
# containers are going to be annoying
# "List" : Function("append", x.sort(), x.sort(), x.sort())(x,y)
ExprRef.__add__ = myadd
s + t

def mymul(x,y):
    match x.sort():
        case Seq:
            return seq_mul(x,y)
        case _:
            print(s.sort().name())
            assert False, s.sort().name()
# containers are going to be annoying
# "List" : Function("append", x.sort(), x.sort(), x.sort())(x,y)
ExprRef.__mul__ = mymul

def myneg(x):
    match x.sort():
        case ArraySort(R,R):
            z = FreshConst(R,prefix="z")
            return Lambda([z], -x[z])
ExprRef.__negate__ = myneg

x + y

In [22]:
#import multipledispatch # But we can't dispatch on the type.. so no go right?
add_dict = {
    Seq: seq_sum,
}
def myadd(x,y):
    return add_dict[x.sort()](x,y)
# what's the point of this indirection though.
def register_add(s, f):
    add_dict[s] = f

ExprRef.__add__ = myadd
s + t

seq_sum(s, t)

In [19]:
repr(s.sort())

'Array(Nat, Real)'

In [ ]:

lim = Function("lim", CSeq, R)
#lim_def = ForAll([s], lim(s) == If(cauchy(s), lim(s.seq), None))
lim_def = Implies(cauchy(s), conv_to(s.seq, lim(s), s.mod))

lim(s + t) == lim(s) + lim(t)
lim(s * t) == lim(s) * lim(t)


# if it converges
infsum = Function("infsum", Seq, R)

Implies(Implies(cauchy(f, Nf), cauchy(g, Ng)), cauchy(add(f,g), Lambda([eps], Nf(eps/2) + Ng(eps/2)))) 


#= cauchy(f,Nf), cauchy(g,Ng) => 
cauchy_eq = zero(sub(g,f), N)

In [ ]:
pow = Function("pow", R, Nat, R)
pow_def = ForAll([x,n], pow(x,n) == If(Nat.is_zero(n), RealVal(1), mul(x, pow(x,Nat.pred(n)))))




# Streams

cvc5 codatatype
continued fractions
digit/bit stream - but bad
+-1 https://www.dcs.ed.ac.uk/home/mhe/plume/node26.html
refining intervals
But hence, refining sets? But a set could converge to multiple points ? Well so could intervals unless you guarantee a sufficient convergence conditions
power of 2 rationals. dyadic?

I have notes squirrelled away somehwere


In [150]:
#https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.continued_fraction.continued_fraction
from sympy.ntheory.continued_fraction import continued_fraction
from sympy import sqrt
continued_fraction((1 + 2*sqrt(3))/5)

[0, 1, [8, 3, 34, 3]]

# Polynomials

Finite polynomials
Vec4 style

SeqSort of coefficients

Harrison
Fundamental theorem of algerba.
Complex numbers. Somehow roots is important?



Finite support
Make an inductive definition not baed on  built in inductions?


finsup(v) == Or(v == vzero, finsup(single(x,y) + w))
There will be models where finsup(ones) is true, but we won't be able to _prove_ its true.

Or

FinSup(v, n, m).wf == ForAll([k], Implies(And(n <= k, m < k), v(k) == 0))





In [70]:
import knuckledragger as kd

def store_reflect(x, n):
    """Keep the array the same, but extract into Store form"""
    acc = x
    for i in range(n):
        acc = kd.smt.Store(acc, i, kd.smt.simplify(x[i]))
    return acc

def store_extract(x, n):
    """zero out non extracted pieces"""
    acc = K(IntSort(), RealVal(0))
    for i in range(n):
        acc = kd.smt.Store(acc, i, kd.smt.simplify(x[i]))
    return acc

def sum_reflect(x,n):
    acc = K(IntSort(), RealVal(0))
    for i in range(n):
        acc = kd.smt.simplify(acc + x[i])
        x = kd.smt.Store(x, i, 0)
    return acc

def sum_extract(p, n):
    """return a sum of monomials"""
    acc = K(IntSort(), RealVal(0))
    for i in range(n):
        acc = kd.smt.simplify(acc + mono(i, p[i]))
    return acc

p = Lambda([n], If(n == 3, RealVal(1), 0))
kd.lemma(reflect(p, 4) == p)
kd.lemma(extract(p, 4) == p)


Store(Store(Store(Store(Lambda(n, If(n == 3, 1, 0)), 0, 0),
                  1,
                  0),
            2,
            0),
      3,
      1)

What but why am I not just working with polynomials?



In [ ]:
converges = Function("converges", RSeq, R, B)
converge_zero = ForAll([x], converges(K(IntSort(), 0), x))
converges_mono = ForAll([p,n,x], converges(mono(n), x)

# formal addition is not convergence dependent
# formal multiplication is.



converge(a,x) & converge(b,x)

# Take value of poweer series to be limsup? Then nonconvergence is eval(x) = Inf
# Or projectivize it? Riemann spehre style?
# supinf = 
#evalp(a,x) = EReal.inf


# Powser
recurrence relations
formal power series
https://www.cs.dartmouth.edu/~doug/powser.html
https://doc.sagemath.org/html/en/reference/power_series/sage/rings/lazy_series.html
cauchy product
solving differential equations in R[[x]]


head(fib_iter(a,b)) = a
tail(fib_iter(a,b)) = fib_iter(a + b, a)

prove fib_iter

List[R] as a generic state?

head([a,b]) = a
tail([a,b]) = [a + b, a]

"Hm - I mean, I think that Nat->Nat can be made isomorphic to NatStream as an F-coalgebra by basically letting head(f)  be f(0), and tail(f) be \x.f(x+1). Is that the sort of thing you're thinking about?" So I could use Lambdas instead of first orderizing my streams?

head(f) = f[0]
tail(f) = Lambda([n], f[n-1])


Use Nat->R as some kind of universe of stream states? Gives us as many tuples as we want...

hd(sum(psum,f)) = psum
tail(sum(psum, f)) = sum(psum + f[0], tail(f))


An online grobner basis over powser could give us the ability to do coefficient matching problems.

https://flintlib.org/applications.html
https://fredrikj.net/python-flint/arb_series.html

https://gitlab.com/feynmanintegrals/fire 
http://ianhenderson.org/similar-rectangles/ hmm addendum at bottom mentions error.
https://github.com/mkauers/ore_algebra/ https://github.com/flintlib/flint/issues/1881
https://gitlab.inria.fr/ricardo-thomas.buring/d-finite-fun
holonomic function = d-finite
https://docs.sympy.org/latest/modules/holonomic/about.html
https://en.wikipedia.org/wiki/Holonomic_function
https://algo.inria.fr/libraries/ algolib maple. gfun. mgfun, regexpcount, 

In [24]:
import knuckledragger as kd
from z3 import *
Powser = IntSort() >> RealSort()
n,m = Ints("n m")
a,b,c = Consts("a b c", Powser)
x,y,z = Reals("x y z")
fP0 = kd.define("P0", [], K(IntSort(), RealVal(0)))
P0 = fP0()
P0.defn = fP0.defn
print(P0.sort())
series = kd.define("series", [x], Store(P0, IntVal(0), x)) #Lambda([n], If(n == 0, x, 0))
add = kd.define("add", [a,b], Lambda([n], a[n] + b[n]))
kd.notation.add.register(Powser, add)
add_comm = kd.lemma(ForAll([a], a + P0 == a), by=[add.defn, P0.defn])
add_assoc = kd.lemma(ForAll([a,b,c], a + (b + c) == (a + b) + c), by=[add.defn])
add_series = kd.lemma(ForAll([x,y], series(x) + series(y) == series(x + y)), by=[add.defn, series.defn, P0.defn])


# tail?
shift = kd.define("shift", [a], Lambda([n], a[n+1])) # Lambda([n], If(n >= 1, a[n + 1], 0))
mul = Function("mul", Powser, Powser, Powser)
kd.notation.mul.register(Powser, mul)
#mul = kd.define("mul", [a,b], Lambda([n], If(n == 0, a[n] * b[n], 
#                                                     shift(a) * b + series(a[0]) * shift(b))))
#mul = kd.define("mul", [a,b], Lambda([n], Sum([m], a[m] * b[n-m]))
# could use smul(a[0], b) instead of series(a[0]) * shift(b)

# a projection
powser = kd.define("powser", [a], Lambda([n], If(m < 0, RealVal(0), a[m])))
is_powser = kd.define("is_powser", [a], powser(a) == a)
powser_idem = kd.lemma(ForAll([a], powser(powser(a)) == powser(a)), by=[powser.defn])

def powser_induct(P):
    return kd.axiom(Implies( ForAll([a], Implies(is_powser(a) & P(a), P(shift(a)))),
                             #----------------------------------
                             ForAll([a], Implies(is_powser(a), P(a)))))




Array(Int, Real)


In [20]:
ArrayRef.__call__ = lambda self, arg: self[arg]

ArraySort(IntSort(), BoolSort(), RealSort())
f = Const("f", ArraySort(IntSort(), BoolSort()))
f(3)

f[3]

In [9]:
# min support
minsup = kd.define("minsup", [a], Lambda([n], Implies(n <= 0, a[n] == 0)))
trim = kd.define("trim", [a], Lambda([n], If(n < 0, 0, a[n])))

minnnonzero


# https://numpy.org/doc/stable/reference/routines.array-creation.html
diff = kd.define("diff", [a], Lambda([n], a[n+1] - a[n]))
cumsum = kd.define_fix("cumsum", [a], Lambda([n], If(n < 0, 0, a[n] + cumsum(a)[n-1])))
zeros = kd.define("zeros", [a], K(IntSort(), RealVal(0)))
ones = kd.define("ones", [a], K(IntSort(), RealVal(1)))



SyntaxError: '(' was never closed (2901492272.py, line 7)

In [23]:
polyval = kd.define("polyval", [a,x], Lambda([n], If(n < 0, 0, a[n] * x**n + polyval(a,x)[n-1]))
pows = kd.define("pows", [x], Lambda([n], If(n < 0, 0, x ** n)) # or recusrive form.




Nth(a, 0)

In [ ]:

# multiplication by keeping a fionite poylnimial eval function?
f0 = kd.define("f0", [], K(IntSort(), RealVal(0)))
f1 = kd.define("f1", [], K(RealSort(), RealVal(1)))
X = kd.define("X", [], Lambda([x], x))
p = f1 + X + X * X

# poly sueqence
PSeq = IntSort() >> (RealSort() >> RealSort())
# The partial sums of prefix polynomials
cumpoly = kd.define("cumpoly", [a], Lambda([n], If(n < 0, f0, Lambda([x], a[n]*x**n + cumpoly(a)[n-1]))))


Polynomial functions. Externally, not syntactically charcterzied by finite diff chain or sampling N points determining the whole function via interpolation.

interp(n,f) = 1 + f(0) + f(1) + ...

interp is a projection from space of all functions to polynomials.

is_poly(f,n) = interp(n,f) == f

In [ ]:
x = Real("x")

In [ ]:
import knuckledragger as kd
from z3 import *

Function("taylor", kd.RFun, kd.RSeq)
Function("converges", kd.RSeq, BoolSort()) # power series vs taylor series
Function("lim", kd.RSeq, RealSort())
Function("CRadius", kd.RSeq, RealSort())



n = Int("n")
recip_n = kd.define("recip_n", [], Lambda([n], If(n == 0, 0, 1/ToReal(n))))
kd.axiom(converges(recip_n))
kd.axiom(lim(recip_n) == 0)

kd.notation.add.define([a,b], Lambda([n], a[n] + b[n]))
converge_add = kd.axiom(kd.QForAll([a,b], converges(a), converges(b), converges(a + b)))
lim_add = kd.axiom(kd.QForAll([a,b], converges(a), converges(b), lim(a + b) == lim(a) + lim(b)))

CRadius

taylor(Lambda([n], If(n < 0, 0, n)))



mul = Function("mul", kd.RSeq, kd.RSeq, RealSort())
kd.notation.mul.register(kd.RSeq, mul)

# Is multiplication for formal laurent series always defined? probably not. Defnitely not.
is_power = kd.define("is_power", [a], QForAll([n], n < 0, a[n] == 0))
pow_proj = kd.define("pow_proj", [a], Lambda([n], If(n < 0, 0, a[n])))
pow_proj_idem = kd.lemma(kd.QForAll([a], pow_proj(pow_proj(a)) == pow_proj(a)))

#monomial = kd.define("monomial", [n], Lambda([m], If(n == m, 1, 0)))
monomial = kd.define("monomial", [n], Store(K(IntSort(), RealVal(0)), n, 1))
p1 = kd.define("p1", [], monomial(0))
X = kd.define("X", [], monomial(1))
# maybe only one has to be a power series for it do be defined?
kd.axiom(kd.QForall([a,b], mul(monomial(n), monomial(n)) == monomial(n + m))) # Implies(n >0, m > 0, ...
const = kd.define("const", [c], Store(K(IntSort(), 0), 0, c)) 
kd.lemma(const(1) == monomial(0))

smul = kd.define("smul", [c,a], Lambda([n], c * a[n])) 
kd.lemma(const(c) * a == smul(c,a))
kd.lemma(smul(c, a + b) == smul(c,a) + smul(c,b))
kd.lemma(smul(c + d, a) == smul(c,a) + smul(d,a))
kd.lemma(smul(c, smul(d,a)) == smul(c*d, a))
kd.lemma(smul(1, a) == a)
kd.lemma(smul(0, a) == const(0))

ExprRef.decl_prop = property(lambda self: self.decl())
ExprRef.children_prop = property(lambda self: self.children())
ExprRef.__match_args__ = ["decl_prop", "children_prop"]
Fn = ExprRef
def poly_norm(t):
    # convert to sympy or convert to z3. call simplify. Get sorted form. Reify back.
    if t.decl() == mul:
        if is_app(t)L
    match t:
        ExprRef(mul, a , b):



# if radius of convergence is >= 1, we can talk about a sequence convergeing to the valuie of the function at x == 1
# it is highly suspect to be directly axiomatizinfg the notion of infinite summation like this. But maybe it's workable.

# Is this well defined?
# We could take mul to be the power seriers trunacating multuiplication = naive_mul(pow_porject(n), pow_project(m))

#kd.axiom(kd.QForall([a,b], (a * b) * c == a * (b * c)))
#kd.axiom(kd.QForall([a,b], a * b == b * a))
#kd.axiom(kd.QForAll([a,b], is_power(a), is_power(b),  a*(b+c) == a*b + a*c))



kd.axiom(kd.QForAll([a,b], converges(a), converges(b), converges(mul(a,b))))









In [26]:
import knuckledragger as kd
from z3 import *
import knuckledragger.theories.Real as R

x,y = Reals('x y')
a,b,c = Consts('a b c', R.RSeq)
n,m = Ints('n m')
kd.notation.add.define([a,b], Lambda([n], a[n] + b[n]))
monomial = kd.define("monomial", [n], Store(K(IntSort(), RealVal(0)), n, 1))
z = monomial(1)
const = kd.define("const", [x], Store(K(IntSort(), RealVal(0)), 0, x))
smul = kd.define("smul", [x,a], Lambda([n], x * a[n]))  # map

mul = Function("mul", R.RSeq, R.RSeq, R.RSeq)
def rmul(self,other):
    if isinstance(other, int) or isinstance(other, float) or other.sort() == RealSort():
        return smul(other,self)
    elif other.sort() == kd.RSeq:
        return mul(self, other)
    assert False
kd.notation.rmul.register(R.RSeq, rmul)
def mul_(self, other):
    if other.sort() == RealSort():
        return smul(other, self)
    elif other.sort() == kd.RSeq:
        return mul(self, other)
kd.notation.mul.register(R.RSeq, rmul)
def radd(self,other):
    if isinstance(other, int) or isinstance(other, float) or other.sort() == RealSort():
        return smul(other,self)
    elif other.sort() == kd.RSeq:
        return mul(other, self)
    assert False

3 * a

3 * a * b + c
kd.utils.simp2(const(10) + z + const(3))
#smul(RealVal(4), (3 * a))



Lambda(n,
       If(0 == n, 10, 0) +
       If(1 == n, 1, 0) +
       If(0 == n, 3, 0))

# numpy

curried Int indexed arrays
Data array + shape

Extraction is interesting.



In [ ]:
import knuckledragger as kd
ndarray = kd.notation.Record("ndarray", ("shape", SeqSort(IntSort())), ("data", ArraySort(IntSort(), RealSort())))

linspace = kd.define("linspace", [a,b,n], ndarray.mk(Unit(IntVal(1)), Lambda([m], a + (b - a) * m / (n - 1))))
zeros = kd.define("zeros", [a], ndarray.mk(a, K(IntSort(), RealVal(0))))
ones = kd.define("ones", [a], ndarray.mk(a, K(IntSort(), RealVal(1))))
eye = kd.define("eye", [n], ndarray.mk(Concat(Unit(n), Unit(n)), Lambda([n], If(i == j, 1, 0))))

# I almost feel like matrix dim match is something people would want.
#dot = kd.define("dot", [x,y], )

# or just code gen a string.
# "lambdify"                
def numpy_extract(x, t):
    if t.decl() == linspace.decl():
        def res(x1):
            np.linspace()
    elif t.decl() == zeros.decl():





BitVecSort()
FiniteDomainSort()
ArraySort(Tuple(Fin(2), Fin(3), ...))

Do broadcasting

sort could be 

V2(V2(V2(x))) could be useful for straitifying big vectors.

It is possible for vector to go wrong at dim 189024, but cmon.
Also maybe we only care about a particular calculation

It's an interesting angle. DOesn't solve everything but could do a lot.
Again, this is kind of in my z3 tutorial. I made a numpy array of z3 objects. Here I make a 

dot can just be blasted.


BitVecs are ArraySort(FinDomain, Bool)


What about just shape on it's own.
Or an asbtract type NDArray I define equations over that I valdiate a bit using concrete stuff.




In [ ]:
class Vec(z3.ArraySortRef):
    def __init__(self, size, sort):
        super().__init__\*args)
        self.shape = args
        self.data = ArraySort(IntSort(), RealSort())
    def __call__(self, *args):
        return self.mk(*args)

# Codatatype / Inf Tree / STream 

What does codatatypes even get you?


define_obs("fib", [], )
a complete set of observations

hd(x), tl(x)



In [ ]:
#RSeq = SeqSort(RealSort())
#a,b,c = Consts("a b c", RSeq)
#hd = kd.define("hd", [a], )
#tail = k.define("tail", [])

# definiing by fiat.
# we don't get rational equivalence for free. Analog of well founedness condition for datatypes. Oddly.

RStream = DeclareSort("RStream")

hd = Function("hd", RStream, RealSort())
tail = Function("tail", RStream, RStream)
zeros = Const("zeros", RStream)

hd_zeros = kd.axiom(hd(zeros) == 0)
tail_zeros = kd.axiom(tail(zeros) == zeros)

fib = Function("fib", RealSort(),RealSort(), RStream)
hd_fib = kd.axiom(ForAll([x,y], hd(fib(x, y)) == x))
tail_fib = kd.axiom(ForAll([x,y], tail(fib(x, y)) == fib(y, x + y)))

# first orderize the rep. Tag in first position.
hd = Function("hd", RSeq, RealSort())
tail = Function("tail", RSeq, RSeq)

fib = kd.define("fib", [a,b], Concat(Unit(0), Unit(a), Unit(b)))
hd_fib = kd.axiom(hd(fib(a,b)) == a)
tl_fib = kd.axiom(tail(fib(a,b)) == fib(b, a + b))

zeros = kd.define("zeros", [], Unit(1))
hd_zeros = kd.axiom(hd(zeros) == 0)
tl_zeros = kd.axiom(tail(zeros) == zeros)

zeros1 = kd.define("zeros1", [], Unit(2))
hd_zeros1 = kd.axiom(hd(zeros1) == 0)
tl_zeros1 = kd.axiom(tail(zeros1) == zeros1)

# They aren't recognized as equal. Hmm.
# An inifnite chain needs something funky (coinduction?), but finite/rational chain shouldn't really.


#hd(zeros) == 0
#tl(zeros) == zero

tails = Function("tails", IntSort(), RSeq())
hds = Function("hds", IntSort(), RealSort())

# fibstate2 = 
#fibstate1 = Function("fibstate", IntSort(), RSeq())
kd.axiom(   fibstate1(n+1) == fibstate1(n) + fibstate2(n))
            fibstate2(n+1) == fibstate1(n)

fibhead = define("fibhead", [n], fibstate0[n] )


In [3]:
from cvc5.pythonic import *
RStream = Datatype('RStream', isCoDatatype=True)
RStream.declare('cons', ('val', RealSort()), ('tail', RStream))
RStream = RStream.create()

zeros, zeros1 = Consts("zeros zeros1", RStream)
s = Solver()
s.add(RStream.val(zeros) == 0)
s.add(RStream.tail(zeros) == zeros)
s.add(RStream.val(zeros1) == 0)
s.add(RStream.tail(zeros1) == zeros1)
s.add(RStream.is_cons(zeros))
s.add(RStream.is_cons(zeros1))
s.add(zeros != zeros1)
assert sat == s.check()
m = s.model()
assert m[zeros].eq(m[zeros1])

In [2]:
Stream = Datatype('Stream', isCoDatatype=True)
Stream.declare('cons', ('car', IntSort()), ('cdr', Stream))
Stream.declare('nil')
Stream = Stream.create()
a = Const('a', Stream)
b = Const('b', Stream)
s = Solver()
s += a == Stream.cons(0, a)
s.check()
sat
s = Solver()
s += a == Stream.cons(0, a)
s += b == Stream.cons(0, b)
s += a != b
s.check()

unsat

In [5]:
from cvc5.pythonic import *
RStream = Datatype('RStream', isCoDatatype=True)
RStream.declare('cons', ('car', IntSort()), ('tail', RStream))
RStream.declare("nil")
RStream = RStream.create()

zeros, zeros1 = Consts("zeros zeros1", RStream)
s = Solver()
s.add(RStream.car(zeros) == 0)
s.add(RStream.tail(zeros) == zeros)
s.add(RStream.car(zeros1) == 0)
s.add(RStream.tail(zeros1) == zeros1)
s.add(RStream.is_cons(zeros))
s.add(RStream.is_cons(zeros1))
s.add(zeros != zeros1)
s.check()
#s.model()
#assert sat == s.check()
#m = s.model()
#assert m[zeros].eq(m[zeros1])

unsat

In [ ]:
ArraySort(IntSort(), BoolSort())

# intermediate value theorem and bisection
mid = (a + b) / 2
bisect = Function("bisect", RFun, RealSort(), RealSort(), RSeq())
bisect(f,a,b) = If(f(mid) == 0, a + b / 2, If(f(mid) < 0, bisect(f, a, mid), bisect(f, mid, b)))
# f(a) < 0, f(b) > 0
# cont(f)

# Eudocus
If floor / real-int interop is such a pain maybe eudoxus is a way to go

# Sup
https://en.wikipedia.org/wiki/Infimum_and_supremum

In [ ]:

RSet = R >> B
RSeq = Nat >> R
sup = Function("sup", RSet, R)
sup2 = Function("sup", RSeq, R)

ubound = Function("upper", RSet, R, B)
ubound_def = ForAll([S, x], ubound(S, x) == ForAll([y], S(y) <= x))

bounded = Function("bounded", RSet, B)
bounded_def = ForAll([S], bounded(S) == Exists([x], ubound(S, x)))

sup_ubound = axiom(QForAll([x], bounded(S), ubound(S, sup(S)))) # it is an upper bound
sup_lub = axiom(QForAll([x, S], ubound(S,x), sup(S) <= x)) # it is the least upper bound

# monotonic saequences mono(s), bounded(s) => sup2(s) = lim(s)


# inf



Infimum and superemum always exist over extended reals?

For monotonic sequences inf/sup = limit

In [ ]:
inf = Function("inf", RSet, R)
inf(SetUnion(A,B)) == min(inf(A), inf(B))
inf(SetIntersect(A,B)) == max(inf(A), inf(B))

inf(A) <= sup(A)

minksum = kd.define("minksum", [A,B], Lambda([z], Exists([x,y], A[x] & B[y] & x + y == z)))
minkprod = Lambda([z], Exists([x,y], A[x] & B[y] & x * y == z))
smul = kd.define("smul", [c,a], Lambda([n], c * a[n]))

lowcut = kd.define("lowcut", [x], Lambda([y], If(y < x, True, False)))
kd.lemma(sup(lowcut(x)) == x)



# Max


In [ ]:
is_max = kd.define("is_max", [f,x], ForAll([y], f[y] <= f[x]))
is_min
is_min(Lambda([x], x*x), 0)





# Derivative Integral
Maybe we hsould have two notions of derivative. The analytic and syntactic version.
deriv() gives something
but
differentiable(f) => is_deriv(f,derive(f))

 https://www.philipzucker.com/z3_diff/

 The "formalderivative"
 The syntactic derivative

In [ ]:
def yoneda(t):
    if num_children() == 2:
        return Lambda([f,g], )
    

kd.lemma(yoneda(f)(id_) == f)

In [ ]:

RFun = ArraySort(R,R)
derive = Function("derive", RFun, RFun) #diff?
sin,cos,exp = Consts("sin cos exp", RFun)

dsin = derive(sin) == cos
dcos = derive(cos) == -sin
dexp = derive(exp) == exp

chain = ForAll([f,g], derive(comp(f,g)) == derive(g) * comp(derive(f),g))
dsum = ForAll([f,g], derive(f + g) == derive(f) + derive(g))
dmul = ForAll([f,g], derive(f * g) == derive(f) * derive(g))
c = Real("c")
dconst = ForAll([c], derive(K(R,c)) == K(R,0))
dx = derive(Lambda([x], x)) == K(R,1)

integ = Function("integ", RFun, RFun)
integ_def = ForAll([f], deriv(integ(f)) == f)
integ_deriv = ForAll([f], Implies(   , integ(deriv(f)) == f))






If one recalls, thr partial application comp(cos,-) is another form of cosine that is also useful. This has something to do with the yoneda lemma if we want to be fancy.



In [ ]:
integ = Function("integ", RFun, R, R, R)
max = Function("max", RFun, R, R, R)
min = Function("min", RFun, R, R, R)

max_def = ForAll([f,a,b], max(f,a,b) == If(a >= b, f(a), f(b)))

max_def = ForAll([f,a,b], max(f,a,b,z) == QForAll([x], And(x <= b, x >= a),  f(x) <= f(z)))

max(f,a,a) == f(a)

max(max(f,a,c), max(f,c,b))
max = Function("max", R, R, R)



integ(f,a,b) = integ(f,a,c) + integ(f,c,b)
integ(f,a,b) <= max(f,a,b) * (b - a)
integ(f,a,b) >= min(f,a,b) * (b - a)



In [ ]:
M = DeclareSort("M") # manifold
RealField = ArraySort(M, R)
d = Function("d", RealField, ??)



In [ ]:


```python
from z3 import *
def lemma(thm,by=[]):
    prove(Implies(And(by), thm))
    return thm

R = RealSort()
RR = ArraySort(R, R)

class RealFun(ArrayRef):
    def __add__(self, other):
        r = plus(self,other)
        r.__class__ = RealFun
        return r

def RealFuns(names):
    xs = Consts(names, RR)
    for x in xs:
        x.__class__ = RealFun
    return xs

z = Real("z")
f,g,h = RealFuns("f g h")
plus = Function("plus", RR, RR, RR)
plus_def = ForAll([f, g], plus(f, g) == Lambda([z], f[z] + g[z]))

plus_comm = lemma(ForAll([f, g], f + g == g + f), [plus_def])
plus_assoc = lemma(ForAll([f, g, h], f + g + h == f + (g + h)), [plus_def])

mul = Function("mul", RR, RR, RR)
mul_def = ForAll([f, g], mul(f, g) == Lambda([z], f[z] * g[z]))

mul_comm = lemma(ForAll([f, g], f * g == g * f), [mul_def])
mul_assoc = lemma(ForAll([f, g, h], f * g * h == f * (g * h)), [mul_def])

x, cos, sin = RealFuns("x cos sin")
x_def = x == Lambda([z],z)



integ = Function("integ", RR, R, R, R)


# Integral

CVould maybe unifromly use 0,1 as integration region.


local substituition foral a<x<=b -> f(x) == g(x) -> integ(f,a,b) == integ(g,a,b)

Or just have it alweays be unbounded intergal, but the bounds go in mu

clamp = 
integ(f, clamp(a,b,mu))


If(x <= a, mu(a), If(x >= b, mu(b), mu(x)))

integ( , -inf, inf)


That riemann stieljes joins up sums is cool. Shadow calc <- stieljes -> calc

discrete ext calc <stieljes?> ext calc
Exterior calcuyluys. int dv = v(1) - v(0)

integ(f, r) <= max(f) dot (r(b) - r(a))

steiljes derivative


(diff(f) == g)   == integ(g,x) == integ(1,f)

integ(f, x) == f + integ()


In [ ]:
integrate = Function("integrate", RFun, RFun, R, R, R) #Reimann stieljes . EReal? Int 1/x
# linear both ways
kd.axiom(integrate(f + g, u, a, b) == integrate(f, u, a, b) + integrate(g, u, a, b))
intergate(const(1), g, a, b) == g(b) - g(a)
integrate(u, v, a, b) == v(a)*u(a) - v(b) * u(b) - integrate(v, u, a, b)
integrate(f, g*h, a, b) == integrate(f*g, h, a, b) + integrate(f*h, g, a, b) # product rule. So the above follows? f == const(1)

#integrate(X, const(1), a , b) == 

intergate1 = kd.define("intergate", [f,a, b], integrate(f , const(1), a, b))

kd.axiom(integrate(smul(c,f), a, b) == integrate(f, a, b) * c)

kd.axiom(integrate(K(R,c), a , b) == (b - a) * c)

In [ ]:
integrate = Function("integrate", RFun, R, R, R) # EReal? Int 1/x
kd.axiom(integrate(f + g, a, b) == integrate(f, a, b) + integrate(g, a, b))
kd.axiom(integrate(smul(c,f), a, b) == integrate(f, a, b) * c)
kd.axiom(integrate(K(R,c), a , b) == (b - a) * c)
u = Lambda([x], integrate(du, c, x))
v = Lambda([x], integrate(dv, d, x))
kd.axiom(ForAll([du,dv],integrate(u * dv, a, b) == u(b) * v(b) - u(a) * v(a) -  integrate(v * du, a, b)))    
kd.axiom(QForAll(ForAll([x], b <= x, x <= a, f(x) <= eps), integrate(f(x), a, b) <= (b - a) * eps)

# The formal derivative. It has no meaning except intergate(fdiff(f), a, b)  == f(a) - f(b)
# The derivative is not a numerically good thing and thus should be viewed with a little suspciion.
# You won't be able to bound fdiff unless you know an actual computation for it
step = If(x <= 0, 0, 1)
fdiff(step)
#fdiff()


https://ocw.mit.edu/courses/res-18-001-calculus-fall-2023/pages/textbook/

f(b) - f(a)


f(a) + f(b) + f(c) >= 





```python

```

Lebesgue integration
Is there numerical version?
What if we wrote a function that outputs the inverse multifunction



If the problem of mosty interest is integration maybe I should attack that directly.The Risch algorithm supposedly does all intergals or something. How could that possibly be true. Maniuplating equations isn't done to my awareness?


```python
%%file /tmp/integ.p

thf(int_decl, type, integ : ($real > $real) > $real).
thf(int_linear, axiom, integ(F + G) = integ(F) + integ(G)).
```

    Overwriting /tmp/integ.p



```python
!zipperposition  /tmp/integ.p
```

    parse error: lexer fails on char +
    
        at file '/tmp/integ.p': line 3, col 31 to 32



# Fixed Point
Fixed point is a bit eaiser to know what we're talking about
- Int * eps = Real
- BitVec * eps = Real bounding

https://soarlab.org/publications/2020_ijcar_bhlnr/#:~:text=Fixed%2Dpoint%20arithmetic%20is%20a,techniques%20or%20reuse%20the%20implementations. An smt theory of fixed point

https://dl.acm.org/doi/10.1145/3524051 Tight Error Analysis in Fixed-point Arithmetic

Yates Randy. 2009. Fixed-point arithmetic: An introduction. http://www.digitalsignallabs.com/fp.pdf

https://theses.hal.science/tel-01158310/file/thesis.pdf Synthesis of certified programs in fixed-point arithmetic,
and its application to linear algebra basic blocks

In [ ]:
floor_eps = kd.define("floor_eps", [x,eps], floor(x/eps) * eps)
floor_eps_bnd = kd.lemma(ForAll([x,eps], And(x - eps <= floor_eps(x,eps), floor_eps(x,eps) <= x, by=[floor_bnd, floor_eps.defn])))

def FixedN(eps):
    R = kd.Record(f"Floor_Eps_{eps}", ("val", kd.R))
    kd.notation.add.define(R, R.mk(floor_eps(x.val + y.val, eps)))
    kd.notation.wf.define([x], floor_eps(x.val, eps) == x.val)
    kd.notation.to_real.register(lambda x: R.val(x))
    #kd.notation.sub.define()
    #kd.notation.mul.define()
    #kd.notation.div.define()





In [33]:
from knuckledragger import *
from z3 import *
Fixed = Datatype("Fixed")
Fixed.declare("mk", ("x", IntSort()), ("eps", RealSort()))
Fixed = Fixed.create()

x = Int("x")
eps = 2**-16
z = Real("z")
fixed2real, fixed2real_def = define("fixed2real", [x], ToReal(x) * eps )
real2fixed, real2fixed_def = define("real2fixed", [z], ToInt(z / eps))



#lemma(ForAll([x], real2fixed(fixed2real(x)) <= 2*eps)
             
             # disparate eps kind of stinks. That starts the be kind of like floating point.
#add = Function("add", Fixed, Fixed, Fixed)
#add_def = ForAll([x,y], real2fixed(fixed2real(x) + fixed2real(y)) == add(x,y))

round, round_def = define("round", [z], fixed2real(real2fixed(z)))


round_down = lemma(ForAll([z], fixed2real(real2fixed(z)) <= z), by=[fixed2real_def, real2fixed_def])
round_up = lemma(ForAll([z], fixed2real(real2fixed(z)) >= z - eps), by=[fixed2real_def, real2fixed_def])


#round_idem = lemma(ForAll([z], round(round(z)) == round(z)), by=[round_def, fixed2real_def, real2fixed_def])
#round_add = lemma(ForAll([x,y], round(x + y) <= round(x) + round(y)), by=[round_def, ]
real_fixed_id = lemma(ForAll([x], real2fixed(fixed2real(x)) == x), by=[real2fixed_def, fixed2real_def])

round_idem = lemma(ForAll([z], round(round(z)) == round(z)), by=[round_def, fixed2real_def, real2fixed_def, real_fixed_id])



#prove(ToInt(ToReal(x)) == x)
#solve(ToInt(RealVal(1.6))== x)


# Float


FPTalks
fpbench
fptools
fpcore
https://fpbench.org/education.html
https://drive.google.com/drive/folders/1HkgMpxi6hsqLSZj9tSnuCxnFExPPjmvx notebooks

https://fpbench.org/community.html toold
https://fpanalysistools.org/


Graded types. rounding error as a monad / computationa effect. Huh
melquiond is nuts

Hmm. For alternating series bound is easy

“Accuracy and Stability of numerical algorithms higha\ham
ATOMU

Pollard
Verified kalman
EKF
1d up to to 3d.
fpbench
adapative
full kalman lu
https://proof.sandia.gov/
Q frameowkr

precisa
calyx. fpcore. oiyutput systemverilog. function coalesiing. Make primitives for wacky regions of dataflow graph

Kahan summation 
https://news.ycombinator.com/item?id=40477604  Taming floating-point sums 
https://www.philipzucker.com/stupid-is-as-stupid-does-floating-point-in-z3py/
https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html
https://stackoverflow.com/questions/32917672/how-to-determine-error-in-floating-point-calculations

https://people.mpi-sws.org/~eva/papers/tacas18_daisy_toolpaper.pdf
Basic flooat model
op(x,y) = op(x,y)(1 + eps) + delta

eps32 = 2^-24, delta32 = 2^-150
eps64 = 2^-53 delta64  = 2^-1024

I guess the analog for fixed point is  op(x,y) = op(x,y) + delta


In [ ]:
x,y,z = Float("x y z")
subnormal
normal


lemma()

In [16]:
from cvc5.pythonic import *
import numpy as np
def lemma(thm, by=[BoolVal(True)], tlimit=1000):
    s = Solver()
    # https://cvc5.github.io/docs/cvc5-1.0.2/resource-limits.html
    # Ahhhhh. tlimit-per, not tlimit. That is very confusing
    s.setOption("tlimit-per", tlimit)
    for hyp in by:
        s.add(hyp)
    s.add(Not(thm))
    res = s.check()
    if res == sat:
        print("countermodel", s.model())
        return s.model()
        assert False, "Failed to prove"
    elif res == unsat:
        print("proved")
        return thm
    else:
        print(res)
        assert False

x,y,z = FPs("x y z", Float32())
float16_plus_comm = lemma(ForAll([x,y], x + y == y + x))
# fails
#float64_plus_assoc = lemma(x + (y + z) == (x + y) + z)

#lemma(x + y == fpRealToFP(RNE(), fpToReal(x) + fpToReal(y), Float32()))

q = Real("q")
q1 = fpRealToFP(RNE(), q, Float32())
eps = np.finfo(np.float32).eps
print(eps)
#lemma(Implies(And(fpIsNormal(q1), q >= 0), fpToReal(q1) <= q*2))

q = Real("q")
q1 = fpRealToFP(RNE(), q, Float64())
#eps = 1/2 * 2**(1-p) q
eps = 2**(-10)
def myabs(x):
    return If(x >= 0, x, -x)
lemma(Implies(And(fpIsNormal(q1), q > 0), (fpToReal(q1) - q) < 100 * eps * q ), tlimit=100000)


proved
1.1920929e-07
unknown


AssertionError: 

In [ ]:
lemma(Implies(And(fpIsNormal(q1), q > 0), (fpToReal(q1) - q) < 100 * eps * q ), tlimit=100000)
None

In [44]:
(1.331814169883728*(2**8) -  11172067/32768) / 

(340.9444274902344, 340.9444274902344)

In [40]:
import numpy as np
counter = 25167869/24576
(counter - np.float32(counter)) / counter

-3.973320116465344e-08

In [58]:
from z3 import *
def lemma(thm, by=[]):
    prove(Implies(And(by), thm))
    return thm

x,y,z = FPs("x y z", Float16())
#float32_plus_comm = lemma(x + y == y + x)
#lemma(x + y == fpRealToFP(RNE(), fpToReal(x) + fpToReal(y), Float16()))

#lemma(Implies(And( #fpIsNormal(x), fpIsNormal(y), 
#                  x >= 0.1, y >= 0.1, x <= 1, y <= 1), 
#                  fpToReal(x + y) <= fpToReal(x) + fpToReal(y) + 2))

#plus_eps = Implies(And())
#lemma(Implies(And(fpIsNormal(x), fpIsNormal(y), fpIsNormal(x + y)), 
#              fpToReal(x + y) <= fpToReal(x) + fpToReal(y) + 2))

q = Real("q")
q1 = fpRealToFP(RNE(), q, Float16())
#eps = np.finfo(np.float16).eps
#lemma(Implies(And(fpIsNormal(q1), q >= 0), fpToReal(q1) <= q*(1 + RealVal(eps)/10000)))

#p = Real("p")
#p1 = fpRealToFP(RNE(), p, Float16())
#lemma(Implies(And( q1 < 1, p1 < 1), 
#              q1 + p1 < 2))

# work with rounding float of reals or real of floats?

#isNormal16_def = lemma(ForAll([x], fpIsNormal(x) == And(fpExp(x) != 0, fpExp(x) != 31)))
lemma(ForAll([x], Not(And(fpIsNormal(x), fpIsSubnormal(x)))))
#lemma(ForAll([x], And(AtMost(fpIsNormal(x), fpIsZero(x), fpIsInf(x), fpIsNaN(x), 1),
#                      Or(fpIsNormal(x), fpIsZero(x), fpIsInf(x), fpIsNaN(x)))))
RD = RoundTowardNegative
RU = RoundTowardPositive

lemma(ForAll([x,y], fpAdd(RU(), x,y) == -fpAdd(RD(), -x,-y,)))
lemma(ForAll([x,y], fpMul(RU(), x,y) == -fpMul(RD(), -x,y)))



proved
proved
proved


ForAll([x, y], fpMul(RTP(), x, y) == -fpMul(RTN(), -x, y))

In [72]:
q = Real("q")
q1 = fpRealToFP(RNE(), q, Float16())
#eps = 1/2 * 2**(1-p) q
eps = 2**(-10)
def myabs(x):
    return If(x >= 0, x, -x)
lemma(Implies(And(q > 1, q < 2), myabs(fpToReal(q1) - q) < myabs(eps * q / 10000000)))

proved


Implies(And(q > 1, q < 2),
        If(fp.to_real(fpToFP(RNE(), q)) - q >= 0,
           fp.to_real(fpToFP(RNE(), q)) - q,
           -(fp.to_real(fpToFP(RNE(), q)) - q)) <
        If((1/1024*q)/10000000 >= 0,
           (1/1024*q)/10000000,
           -((1/1024*q)/10000000)))

failed to solve


In [61]:
eps

0.000977

In [17]:
2**(-10)

0.0009765625

arb, gmp, numpy for bruteforce confirmation of core facts



In [34]:
import numpy as np
float16 = np.frombuffer(np.arange(2**16, dtype=np.uint16).tobytes(), dtype=np.float16)

float16[10000:10010]
# https://en.wikipedia.org/wiki/Half-precision_floating-point_format
print("nan:", np.count_nonzero(np.isnan(float16)))
print("inf:", np.count_nonzero(np.isinf(float16)))
print("neginf:", np.count_nonzero(np.isneginf(float16)))
print("posinf:", np.count_nonzero(np.isposinf(float16)))
print("finite:", np.count_nonzero(np.isfinite(float16)))
print("negative", np.count_nonzero(float16 < 0))
print("index of zeros", np.where(float16 == 0))
print("largest subnormal", float16[0x3ff], float16[0x3ff].tobytes())
print("smallest pos normal", float16[0x400], float16[0x400].tobytes())

nan: 2046
inf: 2
neginf: 1
posinf: 1
finite: 63488
negative 31744
index of zeros (array([    0, 32768]),)
largest subnormal 6.1e-05 b'\xff\x03'
smallest pos normal 6.104e-05 b'\x00\x04'



```racket
;re

; https://en.wikipedia.org/wiki/Machine_epsilon
(define-const eps32 Real (^ 2 -23))



; given eps > 0, abs-bound encodes |x| <= eps as conjunction of linear constraints
(define-fun abs-bound ((x Real) (eps Real)) Bool
    (and
        (<= x eps)
        (<= (- eps) x )
))

(declare-fun rnd-fun (Real) Real)

; Describing operations relationally is nice because you can instantiate properties at use site.
; We describe an overapproximation of the rounding operation
(define-fun rnd ((x Real) (res Real)) Bool
        (and (= res (rnd-fun x)) ; it is a functional relationship
            (abs-bound (- x res) (* eps32 (abs x))) ; with |x - rnd(x)| <= eps32 * |x|
        )
)

; floating point add is Real add then round.
(define-fun fp-add ((x Real) (y Real) (res Real)) Bool
    (rnd (+ x y) res)
)

(define-fun fp-mul ((x Real) (y Real) (res Real)) Bool
    (rnd (* x y) res)
)

; box is useful shorthand for stating initial conditions
(define-fun box ((x Real) (lower Real) (upper Real)) Bool
    (and (<= lower x)
         (<= x upper)
    )
)

(declare-const x Real)
(assert (box x 1 2))
(declare-const y Real)
(assert (box y 1 2))
(declare-const y Real)
(assert (box z 1 2))

(declare-const xy Real)
(assert (fp-add x y xy))
(declare-const xyy Real)
(assert (fp-add xy y xyy))

(declare-const xyz Real)
(assert (fp-mul xy z xyz))
(check-sat)
(get-model)
(eval (abs -1))

(push)
(assert-not (<= (- xyy (+ x y y)) 0.0001))
(check-sat)
(pop)

(push)
(assert-not (>= (- xyy (+ x y y)) -0.00001))
(check-sat)
(pop)

(push)
(assert-not (>= (- xyy (+ x y y)) -0.001))
(check-sat)
(pop)

;(get-model)
;(eval (- xyy (+ x y y)))


(define-fun myprog ((x Real) (y Real) (res Real)) Bool
    (exists ; define local variables
        ((xy Real) (fp73 Real))
     (and ; res := 7/3 * (x + y)
        (fp-add x y  xy)
        (rnd (/ 7 3) fp73)
        (fp-mul fp73 xy res)
     )

    )
)

(define-fun myprog-pre ((x Real) (y Real)) Bool

)

(define-fun myprog-post ((x Real) (y Real) (res Real))
    
)

(assert-not (forall ((x Real) (y Real) (res Real))
            (=> (myprog-pre x y)
                (and (myprog x y res) (myprog-post x y res))
            )
))
```

```racket
;re

(define-fun is-normal ((x Real)) Bool
    (or (and
            (< x (^ 2 300))
            (< (^ 2 -300) x)
         )
        (and
            (< (- x) (^ 2 300))
            (< (^ 2 -300) (- x))
        )
    )
)

(define-fun ulp ((x Real)) Real
   
)

(declare-fun rnd (Real) Real)

(assert (forall ((x Real)) 
    (=> (is-normal x)
         yada
    ) 

    (and (< (- (rnd x) x) (ulp x))
            (< (- (ulp x)) (- (rnd x) x) (ulp x))
))


(assert (forall ((x Real))
    (=>
        (and (< 1 x)
             (< x 2))
        (=  ulp(x))
    )
)


(define-fun-rec ulp ((x Real)) Real
    (ite (and (< x 1) (< x 2)))
         1
    (ite (x < 1)
        (/ (ulp (* x 2) 2)
    (ite (x > 2)
        (* 2 (ulp (/ x 2))
    )
)

;(define-fun fp-mul ((x Real) (y Real))
;    (rnd (* x y)))


; better as a relation?
; relations allow us to bundle in the lemmas we want
; without resorting to forall

; recipe: take a forall axiom, turn it into a define-fun
; This is kind of taking a forall and making it an axiom schema.
; You can still add the forall using (assert (forall ((x )) (my-pred x))
; but it gives the ability to explicitly instantiate

(define-fun rnd-ulp ((x Real)) Bool
    (and (< (- (rnd x) x) (ulp x))
        (< (- (ulp x)) (- (rnd x) x) (ulp x))
)




(define-fun rnd ((x Real) (res Real))
    (and 
        (= res (rnd-fun x))
        (rnd-ulp x)
    )
)


; that these are functions is almost a side feature
; Sometimes the functional nature is useful to the proof.
; probably not that often though

(define-fun ulp ((x Real) (y Real))
    (and
        (= y (ulp-fun x)))
)

(define-fun ulp-def ((x Real)) Bool

)

(define-fun fp-mul ((x Real) (y Real) (z Real))
    (let ((z1 (* x y)))
    (and 
        (= z z1)
        (rnd-ulp z1)
        (ulp-rel z1)

    )

(define-fun fp-sub ((x Real) (y Real) (res Real)) 
    (and
        ...
        (sterbenz x y)
    )

)

```

```racket
(define-sort Var () String)
(define-sort Env () (Array Var Int))
(declare-datatypes ()(
  (Term
    (Var (var1 Var))
    (Lit (lit1 Real))
    (Add (add1 Term) (add2 Term))
    (Mul (add1 Term) (add2 Term))
    (Sub (sub1 Term) (sub2 Term))
    ; And so on
  )
))

(define-fun-rec interp ((e Expr) (rho Env)) Bool
    (match e (
        ((Lit x) x)
        ((Var x) (select rho x))
        ((Add x y) (+ (interp x rho) (interp y rho))

    )
    
    )

)

; outer exists (exists ((rho Env))) is the same as (exists a b c d)
; So we don't get blocked going under an exists.

```

```python
from z3 import *
rnd = Function("rnd", RealSort(), RealSort)
class MyFloat():
    def __init__(self):
        self.x = rnd(FreshReal())
    def __add__(self,rhs):
        return rnd(self.x * rhs.x)

```

Klaus Weihrauch (2000), Computable Analysis.

# COnstructive Real


In [ ]:
# Constructive Real

# Z3 reals are more like algebraic numbers, not reals.
CReal = ArraySort(Nat, RealSort())
eq = Function("eq", CReal, CReal, BoolSort())
ax_eq = trust(ForAll([x,y], eq(x,y) == (ForAll([n], x[n] - y[n] < 1/n))

# Extended Reals
newtype wrapper around Real might be better than the shadow versions. Then I don't have to delete + on the reals.

Could also give myself the extneded reals while I'm at it?

https://en.wikipedia.org/wiki/Extended_real_number_line
https://en.wikipedia.org/wiki/Projectively_extended_real_line
Real = Datatype("Real")
"Nan"
"+Inf"
"-Inf"
Bounded(R)

Maybe 
0+ 
0-
Could also be interesting. Replictae the special behavior of float.
Plus(Real)
Minus(Real) 


In [ ]:
#R = Datatype("R")
#R.declare("wrap", ("real", RealSort()))
#R = R.create()

EReal = Datatype("EReal")
EReal.declare("real", ("val", RealSort()))
EReal.declare("inf")
EReal.declare("neg_inf")
EReal.declare("NaN")
EReal = EReal.create()

undefined = Function("add_undefined", EReal, EReal, EReal)
kd.notation.add.define([x,y], cond(
    (Or(x.is_NaN, y.is_NaN), EReal.NaN), # propagate NaN
    (And(x.is_real, y.is_real),       EReal.mk(x.val + y.val)),
    (And(x.is_inf, Not(y.is_neg_inf)),         EReal.inf),
    (And(x.is_neg_inf, Not(y.is_inf)),         EReal.neg_inf),
    (And(x.is_neg_inf, y.is_neg_inf), EReal.neg_inf))),
    default = EReal.NaN)

kd.notation.sub.define([x,y], add_undefined(x, neg(y)))
kd.notation.mul.define([x,y], add_undefined(x, y))

#kd.le.

kleene_eq = kd.define([x,y], Implies(Not(x.is_NaN), Not(y.is_NaN)))

raw_eq = EReal.__eq__
kd.notation.eq.register(EReal, kleene_eq)

# algerbaic stuff becomes partial. What is inf + neg_inf?


# Smooth Infinitesimal analysis
https://www.cambridge.org/core/books/primer-of-infinitesimal-analysis/B0EF33F73CAF97C180897D2FD0AD1B6E

https://publish.uwo.ca/~jbell/invitation%20to%20SIA.pdf

https://dl.acm.org/doi/10.5555/1983702.1983729 bjorner modellign uitiotinstic inside smt

https://gitlab.inria.fr/fbesson/itauto itauto

https://drops.dagstuhl.de/entities/document/10.4230/LIPIcs.ITP.2021.9

# Autodiff
pair 
Dual(f,f').wf = f(x) ~ f(a) + f'(a)*(x - a)
Question is, what is `~`?
Global bounds are really nice.
local bounds are ok.
I can make a 
rationals can diverge, but then the divergence is kind of...
Extended reals?



epsilon **2 = 0
Autodiff is coefficients of 1 and epsilon.





In [ ]:
class DualFun:
    pass

# Function Records

https://stackoverflow.com/questions/55881078/how-to-know-whether-a-function-is-continuous-with-sympy
https://docs.sympy.org/latest/modules/calculus/index.html#sympy.calculus.util.continuous_domain
https://docs.sympy.org/latest/modules/calculus/index.html#sympy.calculus.util.minimum

fungrim

continuous_domain
singularities

can I get sympy to give me a rigorous upper bound. If differentiable, do it. Then solve for error term. Maximum on domain
I should be using stuff from elsewhere. That's the idea of knuckeldragger. I _have_ to be using the poython ecosystem




There's a lot of data assoiated with real functions. Maybe I need a meta record system to manage it all.



Maybe have everything optional

In [ ]:
@dataclass
class RFun():
    f: R >> R
    dom: SetSort(RealSort())
    codom: SetSort(RealSort())
    lipshitz : RealSort()
    lip_pf : kd.Proof
    modulus_cont : RealSort() >> RealSort()
    modulus_cont_pf : kd.Proof
    deriv : R >> R
    deriv_pf : kd.Proof
    flint_fun :
    eval_method :
    inv : RFun
    float_fun : 
    

    def comp():
        pass
    def add():
        pass





# Topology

Kuratoswki axioms

TPTP had some

In [ ]:
import knuckledragger as kd
RSet = Set(kd.R)
cl = Function("closure", RSet, RSet)
cls_idem = kd.axiom(ForAll([S], cls(cls(S) == cls(S))))
cls_union = kd.axiom(ForAll([S,T], cls(S) + cls(T) == cls(S + T)))
cls_inter = kd.axiom()
clos_mono = kd.axiom(ForAll([S,T], Implies(S <= T, cls(S) <= cls(T))))

compl = kd.define("compl", [S], Lambda([x], Not(S[x])))

half_space = kd.define("half_space", [], Lambda([x], x < 0))
trans = kd.define("trans", [x], Lambda([y], S[y - x]))



close_open = kd.axiom(ForAll([S], cls(S) == cls(S - S)))
interior = Function("interior", RSet, RSet)






# Convex


# Hyperreal

Supposedly, hyperreals are easier to prove stuff in.

The chasing phenomeonon. I am tracking many terms being produced. I could in principle track them all. What if i defined eps and then I can back update it to maiontain constraint. Then program time ~ hyperreal sequence time. Actual reals are constant in time. Other stuff is changing. Hmm. That's fun. Maybe keep calling Z3 To reestablish values. And can keep record. It's not netirely unlike Marshal, except I don't require things to be converging and I keep everyone synced.


Dense linear orfder https://math.stackexchange.com/questions/417900/complete-theories-dense-linear-order https://en.wikipedia.org/wiki/Back-and-forth_method
Big-M and epsilons

Wow, this is going to be a truly ungodly schema

Cofinite. Ultrafilters.
CVC5 has a theory of finite sets. There is a global universe of finite sets, which could be sequence. And then every finset has to be only made of elements in that.
Hmm. That is a bit sticky sounding. This assume we only have finitely many finite sets to talk about.





In [1]:
from z3 import *
import knuckledragger as kd
from knuckledragger import *
from knuckledragger.kernel import Proof, is_proof,axiom
from knuckledragger.utils import open_binder

HReal = DeclareSort("HReal")

class TransferException(Exception):
    pass

def transfer_sort(s : z3.SortRef):
    if s == RealSort():
        return HReal
    elif s == BoolSort():
        return BoolSort()
    # check for arrays? compound sorts?
    else:
        raise TransferException("Unexpected Sort", s)

def transfer_term(t : z3.ExprRef) -> z3.ExprRef:
    assert t.sort() == RealSort() or t.sort() == BoolSort()
    if is_eq(t):
    if is_app(t):
        decl = t.decl()
        f = Function(decl.name(), *[transfer_sort(decl.domain(i)) for i in range(decl.arity())], transfer_sort(decl.range()))
        return f(*[transfer_term(t.arg(i)) for i in range(t.num_args())])
    elif is_quantifier(t) and t.is_forall():
        vs, body = open_binder(t)
        return ForAll([transfer_term(v) for v in vs], transfer_term(body))
    else:
        raise TransferException("Unexpected Term", t)

def transfer_to_hreal(P : Proof) -> Proof:
    assert is_proof(P)
    return axiom(transfer_term(P.thm), by=["transfer", P])

import knuckledragger.theories.Real as Real
transfer_to_hreal(Real.plus_comm)

std = Function("std", HReal, BoolSort())
approx = Function("approx", HReal, HReal, BoolSort())

inf = Function("inf", HReal, BoolSort())



x,y = Consts("x y", HReal)
f = Const("f", ArraySort(HReal, HReal))
cont = define("cont", [f], QForAll([x,y], approx(x,y), approx(f(x), f(y))))



|- ForAll([Y, X], =(plus(X, Y), plus(Y, X)))

In [ ]:
# constructive approach
# hypereals are quotient of Rsequences.
RSeq

# proof relevant cofinite equal. List holds all positions (finite because list ) that the two elements aren't equal
eq = Function("eq", RSeq, RSeq, NatList, BoolSort())
eq_def = ForAll([x,y, n], eq(x,y,n) == ForAll([m], Implies(Not(elem(m,n)), x[m] == y[m])))


# proof relevant finset predicate
finset(s,l) = ForAll([x], s[x] == elem(x,l))
# Exists([l], finset(s,l))

# skolem axiom
# finset(s) == finsetP(s, canon(s))


https://en.wikipedia.org/wiki/Hyperreal_number

Goldblatt

Compactness is important here? Huh.

Adding

Trying to make infinitesimals rigorous

Rings can have things like eps^2 = 0 by taking quotient rings.
Something with R as a subfield.

Constructing a model of hyperreals. Infinite sequences of reals. Quotient by equivlanec e relation of agreeing on cofinite set.

A finite list of disagreement points

```
equiv(a,b,l) = forall n, a[n] = b[n] | n in l 
```

More equational than classical presentation?

R is unique complete ordered field. So R*

Transfer principle

Ultrafilter.
What is a "large" set?

Ultrafilter as a notion of truth. Relation to topos? Modal logic? Intuitionstic logic?

Ultrapower.

Internal set theory

Adding a "standard" predicate to set theory. There is a notion of standard elements vs nonstandard elements.
This has a connection to standard models of constructions like the reals or naturals, which by the weakness of power of description of first order logic, the axims often allow other models besides the standard one.

This has some flavor of the method of "junk" elements used to model partial functions using total functions as I've played around with in category theory.

Nonstandard elements are hard to grasp because whenever you get concrete hold f somthing, it tends (is always?) standard.

I think I've seen it mentioned that maybe "standard" is somehow talking about a fluid boundary of described or construct elements. If I keep around an eps smaller than any number I've considered so far, and maybe go back and fix it up as I go should I ever bring in anything smaller? I do kind of feel like an approximate equality of finite difference is sufficient

Nonstandard models of peano <https://en.wikipedia.org/wiki/Non-standard_model_of_arithmetic>
We can keep adding x > n for any n to the axiom set and stay consistent. The forall n is in the metalogic. By compactness, we can add the forall n into the object logic and stay consistet?

Hypernaturals - a construction of the hypernatural is sequences of naturals identified under a ultrafilter (a "large set"). It's weird that allowing just a little disagreement helps

HyperRationals

Naive Infinitesimals
The reals are a subfield of the hyperreals.
There exist infinittesimals which are smaller than

The "standard" function behaves like a kind of rounding.

Distincting between equality and approximation equality.
a ~ b == st(a) = st(b)

st pushes through regular operations (+, -, ...). So it is a homomorphsim of sorts.
st(eps) = 0

lim = x ~ c /\ x != c -> lim(t(x)) = t(c)
continuity is x ~ y -> f(x) ~ f(y)

In [9]:
from z3 import *
x,y = Ints("x y")
prove(Implies(And(x >= 0, y >= 0, x < 2**32, y < 2**32), x*y < 2**64))

proved


-- cody's proof

Require Import Arith.

Require Import Lia.

Check 2^64.

Search (_^(_ * _)).
Search (_^(_ + _)).

Search (_ * _ < _ * _).

Goal forall a b, a < 2^32 -> b < 2^32 -> a * b < 2^64.
Proof.
  intros a b lt_a lt_b.
  replace (64) with (32 + 32) by reflexivity.
  rewrite Nat.pow_add_r.
  apply Nat.mul_lt_mono_nonneg; lia.
Qed.

In [4]:
import knuckledragger as kd
from knuckledragger.kernel import *
from z3 import *
Z = IntSort()
x,y = Ints("x y")
add = define("add", [x,y], x + y) # hide to avoid cheating
mul = define("mul", [x,y], x * y)


a,b = Ints("a b")
from knuckledragger.utils import calc

#mul_mono = lemma(ForAll([a,b]))

calc(a * b < 2**64,
     a * b < 2**(32 + 32),
     a * b < 2**32 * 2**32)

|- (a*b < 18446744073709551616) == (a*b < 18446744073709551616)

False

# Complex
Angles are a calculus notion?


Maybe the angle addition formula is sufficient.
z1 z1 = r1 r2 cis(theta1 + theta2)

z1 z1_bar = r^2


https://en.wikipedia.org/wiki/CORDIC

Complex analysis should be more than gluing two real analyses together.



In [1]:
import knuckledragger as kd
from z3 import *
C = Datatype("C")
C.declare("mk", ("re", RealSort()), ("im", RealSort()))
C = C.create()
#DatatypeRef.im = property(lambda self: C.im(self))
#DatatypeRef.re = property(lambda self: C.re(self))

z,w,u,z1,z2 = Consts("z w u z1 z2", C)
add = kd.define("add", [z1,z2], C.mk(z1.re + z2.re, z1.im + z2.im))
kd.notation.add.register(C, add)
mul = kd.define("mul", [z1,z2], C.mk(z1.re*z2.re - z1.im*z2.im, z1.re*z2.im + z1.im*z2.re))
kd.notation.mul.register(C, mul)
conj = kd.define("conj", [z], C.mk(z.re, -z.im))


div = kd.define("div", [z1,z2], C.mk((z1.re*z2.re + z1.im*z2.im)/(z2.re**2 + z2.im**2), (z1.im*z2.re - z1.re*z2.im)/(z2.re**2 + z2.im**2)))
kd.notation.div.register(C, div)

C0 = C.mk(0,0)
C1 = C.mk(1,0)

add_zero = kd.lemma(ForAll([z], z + C0 == z), by=[add.defn])
mul_zero = kd.lemma(ForAll([z], z * C0 == C0), by=[mul.defn])
mul_one = kd.lemma(ForAll([z], z * C1 == z), by=[mul.defn])
add_comm = kd.lemma(ForAll([z,w], z + w == w + z), by=[add.defn])
add_assoc = kd.lemma(ForAll([z,w,u], (z + (w + u)) == ((z + w) + u)), by=[add.defn])
mul_comm = kd.lemma(ForAll([z,w], z * w == w * z), by=[mul.defn])


#unstable perfoamnce.
#mul_div = kd.lemma(ForAll([z,w], Implies(w != C0, z == z * w / w)), by=[div.defn, mul.defn], timeout=1000)
##mul_div = Calc()
div_one = kd.lemma(ForAll([z], z / C1 == z), by=[div.defn])
div_inv = kd.lemma(ForAll([z], Implies(z != C0, z / z == C1)), by=[div.defn])

#inv = kd.define("inv", [z], )

# conjugate
# polar
norm2 = kd.define("norm2", [z], z * conj(z))

#kd.lemma(ForAll([z,w], div(z,w) == div(z*conj(w), norm2(w))), by=[div.defn, mul.defn, conj.defn, norm2.defn])

# fundamental theorem... hmm. Z3 does understand real solutions to polys
# https://en.wikipedia.org/wiki/Fundamental_theorem_of_algebra

In [ ]:
lim = Function("lim", ArraySort(C,C), C)
CFun = ArraySort(C,C)
limit00 = kd.define("limit0", [f, eps], [delta], ForAll([x], Implies(x <= delta, f(delta) <= eps)))


In [ ]:
nitpick.register(C, [C0, C1, C.mk(1,1), C.mk(0,1), C.mk(23,8, 100.4), C.mk(-1,0), C.mk(0,0)])

nitpick


# Interval


 
- <https://www.philipzucker.com/z3_diff/>
- <https://www.philipzucker.com/z3-cegar-interval/>
- https://www.philipzucker.com/more-stupid-z3py-tricks-simple-proofs/


- Computational Functional Analysis by Moore <https://www.amazon.com/Computational-Functional-Analysis-Mathematics-Applications/dp/1904275249>
- Introduction to Interval Analysis  <https://epubs.siam.org/doi/10.1137/1.9780898717716> http://www-sbras.nsc.ru/interval/Library/InteBooks/IntroIntervAn.pdf
- methods and applications of inverval analysis <https://epubs.siam.org/doi/book/10.1137/1.9781611970906>
- Interval Analysis: Application in the Optimal Control Problems
- Real Analysis: A Constructive Approach Through Interval Arithmetic - bridger
- Interval Methods for Systems of Equations
- https://fab.cba.mit.edu/classes/S62.12/docs/Hickey_interval.pdf
- bishop
- Constructive functional analysis




In [3]:
Record("TaylorModel", ("f", RFun), ("width", RealSort()))

NameError: name 'Record' is not defined

In [5]:
import flint
flint.arb_series.exp(0)

TypeError: descriptor 'exp' for 'flint.types.arb_series.arb_series' objects doesn't apply to a 'int' object

In [1]:
from knuckledragger import *
import knuckledragger.notation as notation
from knuckledragger.theories.Real import *
#from knuckledragger.theories.Bool import *
from z3 import *
B = BoolSort()
RSet = R >> B

Interval = Datatype("Interval")
Interval.declare("inf") # could also use seminfinite. We'll see. Or could use extended reals.
Interval.declare("mk", ("lo", R), ("hi", R))
Interval = Interval.create()

x,y,z = Reals("x y z")
i,j,k = Consts("i j k", Interval)

# Ok so i could do a SortDisopatch on DatatypeRef overload.
DatatypeRef.hi = property(lambda self: Interval.hi(self))
DatatypeRef.lo = property(lambda self: Interval.lo(self))
DatatypeRef.is_inf = property(lambda self: Interval.is_inf(self))


In [2]:
max = define("max", [x,y], If(x <= y, y, x))
min = define("min", [x,y], If(x <= y, x, y))
abs = define("abs", [x], If(x <= 0, -x, x))

min_comm = lemma(ForAll([x,y], min(x,y) == min(y,x)), by=[min.defn])
min_assoc = kd.lemma(ForAll([x,y,z], min(x,min(y,z)) == min(min(x,y),z)), by=[min.defn])


abs_max = lemma(ForAll([x], abs(x) == max(x,-x)), by=[abs.defn, max.defn])


In [3]:
setof = define("setof", [i], Lambda([x], If(Interval.is_inf(i), BoolVal(True), And(i.lo <= x, x <= i.hi))))

In [4]:
j.is_inf

is(inf, j)

In [5]:
meet = define("meet", [i,j], If(i.is_inf, j,
                                      If(j.is_inf, i,
                                         
                                    Interval.mk(max(i.lo, j.lo), min(i.hi, j.hi)))))


meet_intersect = lemma(ForAll([i,j], 
                           SetIntersect(setof(i), setof(j)) == 
                           setof(meet(i,j))), 
                                by=[setof.defn, meet.defn, min.defn, max.defn])

# DatatypeRef.__and__ = meet

In [6]:
import knuckledragger.notation as notation

notation.le.register(RSet, IsSubset)

In [7]:
join = define("join", [i,j], If(Or(i.is_inf, j.is_inf), Interval.inf, Interval.mk(min(i.lo, j.lo), max(i.hi, j.hi))))

join_union = lemma(ForAll([i,j], 
                    IsSubset(SetUnion(setof(i), setof(j)), setof(join(i,j)))),
                          by=[join.defn, setof.defn, min.defn, max.defn])


In [8]:
# These are not well defined for infinite intervals
width = define("width", [i], i.hi - i.lo)
mid = define("mid", [i], (i.hi + i.lo) / 2)



In [9]:
lt1 = define("lt1", [i,j], And(Not(i.is_inf), Not(j.is_inf), 
                               i.lo <= i.hi, j.lo <= j.hi, 
                               i.hi < j.lo))

# Do i also want interval to be well formed?
lt1_trans = lemma(ForAll([i,j,k], 
                         Implies(And(lt1(i,j), lt1(j,k)),
                                  lt1(i,k))),
                 by=[lt1.defn])

In [10]:
lt2 = define("lt2", [i,j], And(Not(i.is_inf), Not(j.is_inf), 
                            i.lo < j.lo, i.hi < j.hi))

#lt2_subset = lemma(ForAll([i,j], 
#                          Implies(i != j,
#                          lt2(i,j) == IsSubset(setof(i), setof(j)))),
#                          by = [lt2.defn, setof.defn])
lt2_trans = lemma(ForAll([i,j,k], 
                         Implies(And(lt2(i,j), lt2(j,k)),
                                  lt2(i,k))),
                 by=[lt2.defn])


In [11]:
import knuckledragger.notation as notation
add = define("add", [i,j], 
                      If(Or(i.is_inf, j.is_inf), Interval.inf, Interval.mk(i.lo + j.lo, i.hi + j.hi)))

notation.add.register(Interval, add)


add_set = lemma(ForAll([x,y,i,j], Implies(setof(i)[x] & setof(j)[y],  
                                      setof(i + j)[x + y])), 
                                      by=[setof.defn, add.defn])


In [12]:
sub = define("sub", [i,j], 
                      If(Or(i.is_inf, j.is_inf), Interval.inf,
                          Interval.mk(i.lo - j.hi, i.hi - j.lo)))

notation.sub.register(Interval, sub)


sub_set = lemma(ForAll([x,y,i,j], Implies(setof(i)[x] & setof(j)[y],  
                                      setof(i - j)[x - y])), 
                                      by=[setof.defn, sub.defn])

In [26]:
prods = [i.lo *j.lo, i.lo*j.hi, i.hi * j.lo, i.hi*j.hi]
from functools import reduce

# Wow. I don't know about this patchup
mul = define("mul", [i,j], 
                      If(Or(i.is_inf, j.is_inf), Interval.inf,
                      If(And(i.lo > i.hi, j.lo > j.hi), Interval.mk(1,-1),
                      If(i.lo > i.hi, i,
                      If(j.lo > j.hi, j,
                        Interval.mk(reduce(min,prods), reduce(max, prods)))))))

notation.mul.register(Interval, mul)

#mul_set = lemma(ForAll([x,y,i,j], Implies(setof(i)[x] & setof(j)[y],  
#                                      setof(i * j)[x * y])), 
#                                      by=[min.defn, max.defn, setof.defn, mul.defn], timeout=15000)
mul.defn
# should largely follow from monotonicity of mul.

x1 = Real("x1")
mul_pos_mono = lemma(ForAll([x,y,x1], Implies(And(x1 >= x, y >= 0), x1 * y >= x * y)))
mul_neg_mono = lemma(ForAll([x,y,x1], Implies(And(x1 >= x, y <= 0), x1 * y <= x * y)))


       mul(i, j) ==
       If(Or(is(inf, i), is(inf, j)),
          inf,
          If(Or(lo(i) > hi(i), lo(j) > hi(j)),
             mk(1, -1),
             mk(min(min(min(lo(i)*lo(j), lo(i)*hi(j)),
                        hi(i)*lo(j)),
                    hi(i)*hi(j)),
                max(max(max(lo(i)*lo(j), lo(i)*hi(j)),
                        hi(i)*lo(j)),
                    hi(i)*hi(j)))))) to ForAll([i, j],
       mul(i, j) ==
       If(Or(is(inf, i), is(inf, j)),
          inf,
          If(And(lo(i) > hi(i), lo(j) > hi(j)),
             mk(1, -1),
             If(lo(i) > hi(i),
                i,
                If(lo(j) > hi(j),
                   j,
                   mk(min(min(min(lo(i)*lo(j), lo(i)*hi(j)),
                              hi(i)*lo(j)),
                          hi(i)*hi(j)),
                      max(max(max(lo(i)*lo(j), lo(i)*hi(j)),
                              hi(i)*lo(j)),
                          hi(i)*hi(j))))))))


In [14]:


l = Interval.mk(0.9995, 1.0005)
w = Interval.mk(1.9995, 2.0005)



In [30]:
wf = define("wf", [i], Or(i.is_inf, i.lo <= i.hi))

is_real = define("is_real", [i], And(Not(i.is_inf, i.lo == i.hi)))

In [27]:
add_comm = lemma(ForAll([i,j], i + j == j + i), by=[add.defn])
add_assoc = lemma(ForAll([i,j,k], i + (j + k) == (i + j) + k), by=[add.defn])

mul_comm = lemma(ForAll([i,j], i * j == j * i), 
                 by=[mul.defn, max.defn, min.defn]) # or use max_comm, min_comm

#mul_assoc = lemma(ForAll([i,j,k], i * (j * k) == (i * j) * k), by=[mul.defn, max.defn,min.defn])

zero = define("zero", [] , Interval.mk(0,0))
one = define("one", [], Interval.mk(1,1))

add_zero = lemma(ForAll([i], zero() + i == i), by=[zero.defn, add.defn])
mul_one = lemma(ForAll([i], one() * i == i), by=[one.defn, mul.defn, min.defn, max.defn])

In [32]:
subdistr = lemma(ForAll([i,j,k], 
                        Implies(And(wf(i), wf(j), wf(k)),
                        lt2(i * (j + k), i * j + i * k))),
                  by=[mul.defn, wf.defn, add.defn, min.defn, max.defn, lt2.defn ])

LemmaError: (ForAll([i, j, k],
       Implies(And(wf(i), wf(j), wf(k)),
               lt2(mul(i, add(j, k)),
                   add(mul(i, j), mul(i, k))))), 'Countermodel', [knuckledragger_goal = True,
 by_0 = True,
 by_2 = True,
 by_1 = True,
 by_4 = True,
 by_5 = True,
 by_3 = True,
 lo = [mk(1, -1) -> 1,
       mk(3/32, 11/16) -> 3/32,
       mk(-11/32, -3/16) -> -11/32,
       mk(-2, -1/2) -> -2,
       else -> 1/8],
 max = [(11/16, 3/8) -> 11/16,
        (11/16, 11/64) -> 11/16,
        (11/16, 3/32) -> 11/16,
        else -> If(Var(0) + -1*Var(1) <= 0, Var(1), Var(0))],
 add = [(inf, mk(-11/32, -3/16)) -> inf,
        (inf, mk(3/32, 11/16)) -> inf,
        else ->
        If(Or(is(inf, Var(0)), is(inf, Var(1))),
           inf,
           mk(lo(Var(0)) + lo(Var(1)),
              hi(Var(0)) + hi(Var(1))))],
 hi = [mk(1, -1) -> -1,
       mk(3/32, 11/16) -> 11/16,
       mk(-11/32, -3/16) -> -3/16,
       mk(-2, -1/2) -> -1/2,
       else -> 1/2],
 wf = [mk(-2, -1/2) -> True,
  

### Bounding e

There's an incredible slight of hand you can do with interval arithmetic to get rigorous bounds on transcendental functions and differential equations while use pretty generic, computable machinery.

What is euler's constant `e`? Well, there are many definitions, but one natural one is that $y=e^x$ is the function that solves
$\partial_x y = y$

We can write this in integral form
$y(x) = y(0) + \int_0^x y(t)dt$

There is a trick to doing integrals with interval functions.  A simple estimate is to just tuse the minimum and maximum of the function in the region you're evaluating the integral

$\int_a^b y \subseq y * (b-a)$
Or in other words in the integrals $y_{min} * (b-a) <= \int y dt <= y_{max} * (b-a) $

Ok. We can also make an iterative method to get better and better results to the integral equation

$y_{n+1}(x) = y_n(0) + \int_0^x y_n(t)dt$

We can lift this to an interval/tube recurrence relation

Ok. Now, how do we know that this is actually converging to an answer? Well we need to show that the result interval always in strictly inside the previous iteration interval




In [ ]:
integ = define("integ", )


iter = define("iter", [i], 1 + i * (1 - 0))







In [2]:


#def IConsts(xs):
#    xs = Consts(xs, Interval)
#    for x in xs:
#        x.hi = lambda self: Interval.hi(self)
#        x.lo = lambda self: Interval.lo(self)
#    return xs





# Ordering with respect to set inlcusion
le, le_def = define("le", [i,j], If(j.is_inf, BoolVal(True), 
                         If(i.is_inf, BoolVal(False),
                           And(i.hi <= j.hi, j.lo <= i.lo))))

Ordering = Datatype("Ordering")
Ordering.declare("lt")
Ordering.declare("eq")
Ordering.declare("nge")

#ordering with respect to real order
# what do we do with improper
#leR, leR_def = define("le_R", [i,j], If( Or(i.is_inf, j.is_inf), Ordering.nge,  If())



n = Int("n")
pow, pow_def = define("pow", [x,n], 
                                    If(n == 0, 1,
                                    If(n > 0, x * pow(x, n-1), 
                                    If(x == 0, x**n,
                                       1/x * pow(x, n+1)))))

# pow_pow = lemma(ForAll([x,n,m], pow(x,n) == x ** n, by=[pow_def]) # hmm. What about 1/x where x = 0?
#pow_pow = lemma(ForAll([x,n,m], pow(x,n) * pow(x,m) == pow(x, n + m)), by=[pow_def])




add_comm = lemma(ForAll([x,y], i + j == j + i), by=[add_def])
add_assoc = lemma(ForAll([x,y,z], i + (j + k) == (i + j) + k), by=[add_def])
#add_mono_l

lift = define("lift", [x], Interval.mk(x,x))

#lift_mono = define("lift_mono", [f], Lambda([x], lift(f(x)))

square = define("square", [i], If(i.is_inf, Interval.inf, 
                               If(And(i.lo <= 0, i.hi >= 0), Interval.mk(0, max(i.lo*i.lo, i.hi*i.hi)),
                                Interval.mk(min(i.lo*i.lo, i.hi*i.hi), max(i.lo*i.lo, i.hi*i.hi)))))

# sum of squares might be a nice axiom schema.



TypeError: cannot unpack non-iterable FuncDeclRef object

In [68]:
solve((1/x) * (1/x) != 1/(x**2))

[x = 0, /0 = [(1, 0) -> 1/2, else -> 0]]







```python
cos.sort()
```

Array(Real, Real)

Interval analysis is a computational constructive framing

Moore Book
Bridger book
Errett Bishop

Kaucher intervals <https://arxiv.org/pdf/2202.03058>

Fixed point theorems.
An uninterpreted sort is not that bad for non recursive datatypes.

```python
from z3 import *

I = DeclareSort("Interval")
a, b, c = Consts("a b c", I)
R = RealSort()
x,y,z = Reals("x y z")
low = Function("low", I, R)
high = Function("high", I, R)
ForAll([a], low(a) <= high(a))

sub = Function("sub", I, I, BoolSort())
sub_def = ForAll([a,b], sub(a,b) == And(low(a) > low(b), high(a) < high(b)))

sub_irrefl = ForAll([a], Not(sub(a,a)))
prove(Implies(sub_def, sub_irrefl))

sub_asym = ForAll([a,b], Implies(sub(a,b), Not(sub(b,a))))
prove(Implies(sub_def, sub_asym))

sub_trans = ForAll([a,b,c], Implies(And(sub(a,b), sub(b,c)), sub(a,c)))
prove(Implies(sub_def, sub_trans))

subeq = Function("subeq", I, I, BoolSort())
subeq_def = ForAll([a,b], subeq(a,b) == Or(sub(a,b), a == b))

subeq_refl = ForAll([a], subeq(a,a))
prove(Implies(subeq_def, subeq_refl))

subeq_asym = ForAll([a,b], Implies(And(subeq(a,b), subeq(b,a)), a == b))
prove(Implies(And(subeq_def, sub_asym), subeq_asym))

subeq_trans = ForAll([a,b,c], Implies(And(subeq(a,b), subeq(b,c)), subeq(a,c)))
prove(Implies(And(subeq_def, sub_trans), subeq_trans))

# An interval can be dominated by another interval. This is distinct from subset
lt = Function("lt", I, I, BoolSort())
lt_def = ForAll([a,b], lt(a,b) == high(a) < low(b))


mono = Function("mono", ArraySort(I,I), BoolSort())
f = Array("f", I, I)
mono_def = ForAll([f], mono(f) == ForAll([a], sub(a,f[a])))

fix = Function("fix", ArraySort(I,I), I)
fix_def = ForAll([f], Implies(mono(f), fix(f) == f[fix(f)]))

# too dangerous to skolemize?
int_exists = ForAll([x,y], Implies( x <= y, Exists([a], And(low(a) == x, high(a) == y))))

iadd = Function("iadd", I, I, I)

iadd_comm = ForAll([x,y], iadd(x,y) == iadd(y,x))
iadd_assoc = ForAll([x,y,z], iadd(x,iadd(y,z)) == iadd(iadd(x,y),z))
iadd_mono = ForAll([x,y,z], Implies(sub(x,y), sub(iadd(x,z),iadd(y,z))))

iadd_high = ForAll([a,b], high(iadd(a,b)) == high(a) + high(b))
iadd_low = ForAll([a,b], low(iadd(a,b)) == low(a) + low(b))

iadd_monol = ForAll([a], mono(Lambda[a], iadd(a,b)))
```


```souffle

.type interval = [l : float, u : float]
.decl meet(x : interval , y : interval, z : interval) inline
meet([lx,ux],[ly,uy],[max(lx,ly), min(ux,uy)]) :- true.

.decl join(x : interval , y : interval, z : interval) inline
join([lx,ux],[ly,uy],[min(lx,ly), max(ux,uy)]) :- true.

.decl add(x : interval , y : interval, z : interval) inline
add([lx,ux],[ly,uy],[lx + ly, ux + uy]) :- true.

.decl sub(x : interval , y : interval, z : interval) inline
sub([lx,ux],[ly,uy],[lx - uy, ux - ly]) :- true.

.decl mul(x : interval , y : interval, z : interval) inline
mul([lx,ux],[ly,uy],[min(a,b,c,d), max(a,b,c,d)]) :- a = lx * ly, b = ux * ly, c = lx * uy, d = ux * uy.

.decl abs(x : interval , y : interval) inline
abs([lx,ux],[lx,ux]) :- lx > 0.
abs([lx,ux],[-ux, -lx]) :- ux < 0.
abs([lx,ux],[0, max(-lx, ux)]) :- lx <= 0, ux >= 0.


.decl test(x : interval)
test(z) :- mul([2,4], [4,5],z).
.output test(IO=stdout)


.type Expr = 
      Add {x : Expr, y : Expr}
    | Sub {x : Expr, y : Expr}
    | Mul {x : Expr, y : Expr}
    | Rnd {x : Expr}
    | Lit {x : float}
    | Var {x : symbol}

// Expression e is in interval i
.decl BND(e : Expr, i : interval)
// Absolute value of e is in interval i
.decl ABS(e : Expr, i : interval) 
// x = y * (1 + e) 
.decl REL(x : Expr, y : Expr, i : interval)
// 
//.decl FIX(x : Expr, k)

// The value of expression x is not zero.
.decl NZR(x : Expr)
.decl EQL(x : Expr, y : Expr) eqrel

EQL(x,y) :- BND(x, [a,a]), BND(y, [a,a]).
NZR(x) :- BND(x, [a,b]), (a > 0 ; b < 0).

.decl exprs(e : Expr)
EQL(x,x) :- exprs(x).

BND(t, [a,a]) :- exprs(t), t = $Lit(a).
BND(t, [0,0]) :- exprs(t), t = $Sub(a, a).

BND(z, iz) :- exprs(z), BND(x,ix), BND(y,iy), (
            (z = $Add(x,y), add(ix,iy,iz));
            (z = $Sub(x,y), sub(ix,iy,iz));
            (z = $Mul(x,y), mul(ix,iy,iz))).

ABS(e, iy) :- BND(e, ix), abs(ix,iy).


```



# Manifolds

title: Manifolds in Equational Logic


Automated reasoning over
Matroids. Synthetic theory of linear independence <https://en.wikipedia.org/wiki/Matroid>
universal injective
a dot b = a dot c !-> b = c

"represented by" =o=  observer dependent
two basic vectors e(0),e(1)
e'(1) e'(2) a different basis.

A parametrized set in abstract V
v : R R R -> V
v(20,30,-3)
It's a linear reprsentation / map
v(X1,Y1) + v(X2,Y2) = v(X1 + X2, Y1 + Y2)

dot(v(X,Y), v(W,Z)) = X*W + Y*Z

<https://en.wikipedia.org/wiki/Vector_space>


Interesting questions: Is a coordinate expression an invarinant?  f(X,Y,Z) ?= f'(p(X,Y,Z)). Converting coordinate expressions to invariant forms.
axiomatic minkowski

I don't know that there can be a canonical formulation of any logic.

But equational logic is trees of symbols, some of which are variable symbols. The 0-arity constant symbol `x` is very different from the variable `X` `?x`.
This distinction is useful. A statement `f(x) = 1` is a statement about the interaction of a particular `f` on a particular `x`. This may be an equation we'd like to solve for possibille values of `x`. `f(X) = 1` however is a universal statement. If we were working in forist order logic, which throws exists and forall quantifiers into the mix, this statement is the analog of `forall x, f_fol(x) =_fol 1_fol`. It is now stating nothing about a particular `x` but instead stating a global property of the function `f`, that `f` is the constant function.

Coordinate systems are a tricky concept. The notation is quite slippery and you learn your way around it.

Coordinates are an address book for points. They parametrize points such that.
In other words a coordinate system *is* a function of the type `R^n -> M`

The concept of a point is discussed in the textbooks, but it is an abstract entity. Since it doesn't really have calculational properties the way numbers, matrices, and numerical functions do. Even abstract things like quantum operators, kind of we have some matrix implementation in the back of my head. Points kind of have no operations on them in general. They can be in relation to other entities like curve, surfaces, etc, but there isn't a notion of point multiplication or addition or scaling except in very special spaces.

There is a related notion for a coordinate system of `x : M -> R`.

A coorindate transformation such as translation is written like `x' = x + a`. What are these `x`? This is certainly nonsensical as `X' = X + a`. It is also not exactly that we mean are talking about a particular `x` or `x'`. This might be so if we had a particular point of interest, `x'(p) = x(p) + a` but we are talking about transforming the coordinate *system* everywhere.
Actually what we are saying is that for any point `x'(P) = x(P) + a`.

Anyway, I think the appropriate notation here is

```
coord'(X,Y,Z) = coord(X - a, Y, Z)
```

Again, we get used to that if you have an expression `f(x')`,

```bash
cart : R^3 -> M
cart(_,_,_)

x y z : M -> R

x(cart(X,Y,Z)) = X
y(cart(X,Y,Z)) = Y

cyl(R,T,Z) = cart(Rcos(T), Rsin(T), Z)
cart(X,Y,Z) = cyl(sqrt(X^2 + Y^2), atan2(X,Y), Z)

point : M

dist : M -> M -> R
dist(P1,P2) 

% hyperreals?


a curve.
p : R -> M
p(T) = cart(T,T,0)


%vector derivative
% a rate of change of a function
dp : (M -> R) -> R -> R  % (?)
dp(f)(T) = f'(p(T)) * 2

% vector field
v : (M -> R) -> M -> R




a surface
q : M2 -> M

chart : R^2 -> M2
x(q(chart(X,Y))) =

% exterior deriavtive
d : (M -> R) -> M -> ?

% a different cartersian coordinate system
x(cart'(X,Y,Z)) = x(cart(X,Y,Z)) + a
```

it would be nice for the charts to be partial.
